In [2]:
# Install ace-tools and other necessary packages
!pip install ace-tools

# Import libraries for Google Colab and data science tasks
from google.colab import drive
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
import os
import sys

# Attempt to import the ace-tools library.
try:
    import ace_tools
    print("ace_tools imported successfully.")
except ModuleNotFoundError as e:
    print("Error importing ace_tools:", e)
    print("It appears that ace-tools was installed, but the module 'ace_tools' was not found.")
    print("If the package is intended for use as an importable module, please verify the correct import name in its documentation.")

# Mount Google Drive to access files stored in your Drive
drive.mount('/content/drive')


Error importing ace_tools: No module named 'ace_tools'
It appears that ace-tools was installed, but the module 'ace_tools' was not found.
If the package is intended for use as an importable module, please verify the correct import name in its documentation.
Mounted at /content/drive


In [ ]:
import pandas as pd

# Define the input file path for your Final 25 Excel file.
file_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Final 25.xlsx'

# Read the Excel file into a DataFrame.
df = pd.read_excel(file_path)

# Extract the columns of interest and remove duplicate PERMNO entries.
unique_data = df[['PERMNO', 'Ticker', 'Company', 'Sector']].drop_duplicates(subset='PERMNO', keep='first')

# Display the resulting DataFrame.
print(unique_data)

# Define the output file path where the new Excel file will be saved.
output_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Company 1.xlsx'

# Save the processed DataFrame as an Excel file.
unique_data.to_excel(output_path, index=False)
print(f"Data successfully saved to {output_path}")


        PERMNO Ticker                          Company            Sector
0        10104   ORCL                      ORACLE CORP        Technology
252      10107   MSFT                   MICROSOFT CORP        Technology
504      10138   TROW           T ROWE PRICE GROUP INC        Financials
756      10145    HON      HONEYWELL INTERNATIONAL INC       Industrials
1008     10225     FO               FORTUNE BRANDS INC  Consumer Staples
...        ...    ...                              ...               ...
145011   92528    AHC                    A H BELO CORP        Healthcare
145200   92602     PM  PHILIP MORRIS INTERNATIONAL INC  Consumer Staples
145388   92611      V                         VISA INC        Technology
145576   92655    UNH           UNITEDHEALTH GROUP INC        Healthcare
145828   93091    AOL                        A O L INC     Communication

[483 rows x 4 columns]
Data successfully saved to /content/drive/My Drive/Colab Notebooks/823 Project/Company 1.xlsx


In [ ]:
# Install XlsxWriter if not already installed.
!pip install XlsxWriter

import pandas as pd
import xlsxwriter

# ---------------------------
# Step 1. Read the Excel File
# ---------------------------
input_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Final 25.xlsx'
df = pd.read_excel(input_path)

# ---------------------------
# Step 2. Create Timestamp Column
# ---------------------------
# Assumes that the file has 'Year' and 'Month' columns.
# The Timestamp column is created in YYYYMM format.
df['Timestamp'] = df['Year'].astype(str) + df['Month'].apply(lambda x: f"{int(x):02d}")

# ---------------------------------------------------------
# Step 3. Group by Timestamp, sort by Return_Next 3M, select Top 30, and assign rank.
# ---------------------------------------------------------
grouped_data = []
for timestamp, group in df.groupby('Timestamp'):
    # Sort the group by 'Return_Next 3M' in descending order (highest returns first)
    group_sorted = group.sort_values(by='Return_Next 3M', ascending=False).head(30).copy()

    # Reset index and assign a unique rank from 1 to number of rows
    group_sorted = group_sorted.reset_index(drop=True)
    group_sorted['Actual Rank'] = range(1, len(group_sorted) + 1)

    # Keep only the necessary columns in the specified order
    group_sorted = group_sorted[['Timestamp', 'Actual Rank', 'Return_Next 3M', 'PERMNO', 'Ticker', 'Company', 'Sector']]

    grouped_data.append((timestamp, group_sorted))

# ---------------------------------------------------------
# Step 4. Write to a new Excel file with group headers for each Timestamp.
# ---------------------------------------------------------
output_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 1.xlsx'

# Create a workbook and add a worksheet using xlsxwriter.
# The 'nan_inf_to_errors' option converts any NaN/INF to Excel error values.
workbook = xlsxwriter.Workbook(output_path, {'nan_inf_to_errors': True})
worksheet = workbook.add_worksheet("Ranking")

# Define the standard data column headers.
columns = ["Timestamp", "Actual Rank", "Return_Next 3M", "PERMNO", "Ticker", "Company", "Sector"]

# Start writing at the first row.
current_row = 0

# Process groups sorted by Timestamp.
for timestamp, group_df in sorted(grouped_data, key=lambda x: x[0]):
    # Write a header row for the group.
    header_text = f"Top 30 Stocks of {timestamp}"
    worksheet.write(current_row, 0, header_text)
    current_row += 1

    # Write the column headers.
    for col_idx, col_name in enumerate(columns):
        worksheet.write(current_row, col_idx, col_name)
    current_row += 1

    # Write each row of the current group.
    for _, row_data in group_df.iterrows():
        for col_idx, col_name in enumerate(columns):
            worksheet.write(current_row, col_idx, row_data[col_name])
        current_row += 1

    # Optional: Insert a blank row between groups for better readability.
    current_row += 1

# Finalize and close the workbook.
workbook.close()
print(f"Data successfully saved to {output_path}")


Data successfully saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 1.xlsx


In [ ]:
# Install XlsxWriter if not already installed.
!pip install XlsxWriter

import pandas as pd
import xlsxwriter

# ---------------------------
# Step 1. Read the Final 34 File
# ---------------------------
final34_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Final 34.xlsx'
df34 = pd.read_excel(final34_path)

# ---------------------------
# Step 2. Merge in Company Info from Company 1 File
# ---------------------------
company1_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Company 1.xlsx'
df_company = pd.read_excel(company1_path)

# Merge on PERMNO to bring in Ticker, Company, and Sector (Company 1 file is assumed to have unique PERMNO values)
df34 = pd.merge(df34, df_company[['PERMNO', 'Ticker', 'Company', 'Sector']], on='PERMNO', how='left')

# ---------------------------
# Step 3. Group by Timestamp and Rank Top 30 Winners per Month
# ---------------------------
# For each Timestamp group, sort descending by 'Predicted_Return_Next_3M',
# select the top 30, and assign a unique rank from 1 to 30.
grouped_data = []
for timestamp, group in df34.groupby('Timestamp'):
    group_sorted = group.sort_values(by='Predicted_Return_Next_3M', ascending=False).head(30).copy()
    group_sorted = group_sorted.reset_index(drop=True)
    group_sorted['RollingOLS Non-scaled Rank'] = range(1, len(group_sorted) + 1)

    # Keep only the required columns in the specified order.
    group_sorted = group_sorted[['Timestamp', 'RollingOLS Non-scaled Rank', 'Predicted_Return_Next_3M',
                                   'PERMNO', 'Ticker', 'Company', 'Sector']]

    grouped_data.append((timestamp, group_sorted))

# ---------------------------
# Step 4. Write to a new Excel file ("Ranking 2.xlsx") with Group Headers
# ---------------------------
output_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 2.xlsx'

# Create a workbook with the 'nan_inf_to_errors' option to handle any NaN/INF values.
workbook = xlsxwriter.Workbook(output_path, {'nan_inf_to_errors': True})
worksheet = workbook.add_worksheet("Ranking")

# Define the column headers for the data rows.
columns = ["Timestamp", "RollingOLS Non-scaled Rank", "Predicted_Return_Next_3M",
           "PERMNO", "Ticker", "Company", "Sector"]

current_row = 0

# Process each group sorted by Timestamp.
for timestamp, group_df in sorted(grouped_data, key=lambda x: x[0]):
    # Write a header row for this group.
    header_text = f"Top 30 Stocks of {timestamp}"
    worksheet.write(current_row, 0, header_text)
    current_row += 1

    # Write the column headers.
    for col_idx, col_name in enumerate(columns):
        worksheet.write(current_row, col_idx, col_name)
    current_row += 1

    # Write each row of data for this Timestamp group.
    for _, row_data in group_df.iterrows():
        for col_idx, col_name in enumerate(columns):
            worksheet.write(current_row, col_idx, row_data[col_name])
        current_row += 1

    # Insert a blank row between groups for readability.
    current_row += 1

# Finalize and close the workbook.
workbook.close()
print(f"Data successfully saved to {output_path}")


Data successfully saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 2.xlsx


In [ ]:
# Install required packages if not already installed.
!pip install XlsxWriter openpyxl

import pandas as pd
import xlsxwriter

def parse_ranking_file(filepath):
    """
    Parses a ranking file written with group headers.
    Each group starts with a cell in column A containing "Top 30 Stocks of YYYYMM",
    followed by a row of column headers and then the data rows.
    Returns a dictionary with keys as the Timestamp (YYYYMM) and values as DataFrames.
    """
    # Read the file without a fixed header.
    df = pd.read_excel(filepath, header=None, engine='openpyxl')
    groups = {}
    current_timestamp = None
    current_columns = None
    data_rows = []

    for _, row in df.iterrows():
        cell_val = str(row[0]) if pd.notnull(row[0]) else ""

        if cell_val.startswith("Top 30 Stocks of "):
            # If a previous group exists, save its data.
            if current_timestamp is not None and current_columns is not None and data_rows:
                groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
            # Start a new group; extract the timestamp.
            current_timestamp = cell_val.split("Top 30 Stocks of ")[1].strip()
            current_columns = None
            data_rows = []
        elif current_timestamp is not None and current_columns is None:
            # This row is the column headers.
            current_columns = row.dropna().tolist()
        elif current_timestamp is not None and current_columns is not None:
            # Check if the row is not entirely empty.
            if not row.isnull().all():
                row_data = row.tolist()[:len(current_columns)]
                data_rows.append(row_data)
    # Save the last group.
    if current_timestamp is not None and current_columns is not None and data_rows:
        groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)

    return groups

# ---------------------------
# Step 1. Read the Ranking Files
# ---------------------------
ranking1_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 1.xlsx'
ranking2_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 2.xlsx'

ranking1_groups = parse_ranking_file(ranking1_path)
ranking2_groups = parse_ranking_file(ranking2_path)

# ---------------------------
# Step 2. Combine Groups with Matching Timestamps Side by Side
# ---------------------------
common_timestamps = sorted(set(ranking1_groups.keys()).intersection(set(ranking2_groups.keys())))
combined_groups = {}

for ts in common_timestamps:
    df1 = ranking1_groups[ts].reset_index(drop=True)
    df2 = ranking2_groups[ts].reset_index(drop=True)
    # Concatenate horizontally: Ranking 2 columns follow Ranking 1 columns.
    combined_df = pd.concat([df1, df2], axis=1)
    combined_groups[ts] = combined_df

# ---------------------------
# Step 3. Write the Combined Data to "Ranking 3.xlsx"
# ---------------------------
output_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 3.xlsx'
workbook = xlsxwriter.Workbook(output_path, {'nan_inf_to_errors': True})
worksheet = workbook.add_worksheet("Combined Ranking")

current_row = 0
for ts in common_timestamps:
    # Write the group header.
    header_text = f"Top 30 Stocks of {ts}"
    worksheet.write(current_row, 0, header_text)
    current_row += 1

    # Retrieve the combined DataFrame.
    combined_df = combined_groups[ts]

    # Write column headers.
    for col_idx, col_name in enumerate(combined_df.columns):
        worksheet.write(current_row, col_idx, col_name)
    current_row += 1

    # Write the data rows.
    for _, row_data in combined_df.iterrows():
        for col_idx, cell in enumerate(row_data):
            worksheet.write(current_row, col_idx, cell)
        current_row += 1

    # Insert a blank row between groups.
    current_row += 1

workbook.close()
print(f"Combined data successfully saved to {output_path}")


Combined data successfully saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 3.xlsx


In [ ]:
# Install XlsxWriter if needed.
!pip install XlsxWriter

import pandas as pd
import xlsxwriter

# ---------------------------
# Step 1. Read and Filter the Final 25 File
# ---------------------------
input_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Final 25.xlsx'
df = pd.read_excel(input_path)

# Exclude rows from sectors: Real Estate, Financials, Healthcare.
df = df[~df['Sector'].isin(['Real Estate', 'Financials', 'Healthcare'])]

# ---------------------------
# Step 2. Create a Timestamp Column in YYYYMM Format
# ---------------------------
# Assumes the file has 'Year' and 'Month' columns.
df['Timestamp'] = df['Year'].astype(str) + df['Month'].apply(lambda x: f"{int(x):02d}")

# ---------------------------
# Step 3. Remove Duplicate Entries
# ---------------------------
# Ensure each row is unique with respect to PERMNO and Return_Next 3M.
df = df.drop_duplicates(subset=['PERMNO', 'Return_Next 3M'])

# ---------------------------
# Step 4. Group by Timestamp, Rank Top 30 Winners, and Assign Actual Rank
# ---------------------------
grouped_data = []

# Group the DataFrame by the new Timestamp column.
for timestamp, group in df.groupby('Timestamp'):
    # Sort the group by 'Return_Next 3M' in descending order (highest returns first)
    group_sorted = group.sort_values(by='Return_Next 3M', ascending=False).head(30).copy()

    # Reset the index and assign a unique rank from 1 to len(group_sorted)
    group_sorted = group_sorted.reset_index(drop=True)
    group_sorted['Actual Rank'] = range(1, len(group_sorted) + 1)

    # Select only the required columns.
    group_sorted = group_sorted[['Timestamp', 'Actual Rank', 'Return_Next 3M', 'PERMNO', 'Ticker', 'Company', 'Sector']]

    grouped_data.append((timestamp, group_sorted))

# ---------------------------
# Step 5. Write the Result to "Ranking 4.xlsx" with Group Headers
# ---------------------------
output_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 4.xlsx'

# Create a new workbook with the 'nan_inf_to_errors' option to handle any NaN/INF values.
workbook = xlsxwriter.Workbook(output_path, {'nan_inf_to_errors': True})
worksheet = workbook.add_worksheet("Ranking")

# Define the column headers.
columns = ["Timestamp", "Actual Rank", "Return_Next 3M", "PERMNO", "Ticker", "Company", "Sector"]

current_row = 0

# Process each group sorted by Timestamp.
for timestamp, group_df in sorted(grouped_data, key=lambda x: x[0]):
    # Write a header row for the group.
    header_text = f"Top 30 Stocks of {timestamp}"
    worksheet.write(current_row, 0, header_text)
    current_row += 1

    # Write the column headers.
    for col_idx, col_name in enumerate(columns):
        worksheet.write(current_row, col_idx, col_name)
    current_row += 1

    # Write each row of data for this Timestamp group.
    for _, row_data in group_df.iterrows():
        for col_idx, col_name in enumerate(columns):
            worksheet.write(current_row, col_idx, row_data[col_name])
        current_row += 1

    # Insert a blank row between groups for readability.
    current_row += 1

# Finalize and close the workbook.
workbook.close()
print(f"Ranking 4 successfully saved to {output_path}")


<ipython-input-11-c2d02cb2760a>:20: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Timestamp'] = df['Year'].astype(str) + df['Month'].apply(lambda x: f"{int(x):02d}")


Ranking 4 successfully saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 4.xlsx


In [ ]:
# Install XlsxWriter if not already installed.
!pip install XlsxWriter

import pandas as pd
import xlsxwriter

# ---------------------------
# Step 1. Read the Final 34 File
# ---------------------------
final34_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Final 34.xlsx'
df34 = pd.read_excel(final34_path)

# ---------------------------
# Step 2. Exclude Rows Based on One-Hot Encoded Sector Dummies
# ---------------------------
# Exclude rows where any of these dummy variables is 1.
df34_filtered = df34[(df34['Sector_Real Estate'] != 1) &
                     (df34['Sector_Financials'] != 1) &
                     (df34['Sector_Healthcare'] != 1)]

# ---------------------------
# Step 3. Remove Duplicate Entries per Timestamp and PERMNO
# ---------------------------
# For companies with multiple rows in the same month (Timestamp), keep the one with the highest Predicted_Return_Next_3M.
df34_filtered = df34_filtered.sort_values('Predicted_Return_Next_3M', ascending=False)
df34_unique = df34_filtered.groupby(['Timestamp', 'PERMNO'], as_index=False).first()

# ---------------------------
# Step 4. Merge in Company Info from the Company 1 File
# ---------------------------
company1_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Company 1.xlsx'
df_company = pd.read_excel(company1_path)

# Merge to bring in Ticker, Company, and Sector information.
df34_merged = pd.merge(df34_unique,
                       df_company[['PERMNO', 'Ticker', 'Company', 'Sector']],
                       on='PERMNO',
                       how='left')

# ---------------------------
# Step 5. Group by Timestamp, Rank Top 30 Winners, and Assign Rank
# ---------------------------
grouped_data = []

# For each Timestamp group, sort descending by Predicted_Return_Next_3M and select top 30.
for timestamp, group in df34_merged.groupby('Timestamp'):
    group_sorted = group.sort_values(by='Predicted_Return_Next_3M', ascending=False).head(30).copy()
    group_sorted = group_sorted.reset_index(drop=True)
    group_sorted['RollingOLS Non-scaled 1 Rank'] = range(1, len(group_sorted) + 1)

    # Keep only the required columns.
    group_sorted = group_sorted[['Timestamp', 'RollingOLS Non-scaled 1 Rank',
                                   'Predicted_Return_Next_3M', 'PERMNO',
                                   'Ticker', 'Company', 'Sector']]

    grouped_data.append((timestamp, group_sorted))

# ---------------------------
# Step 6. Write the Result to "Ranking 5.xlsx" with Group Headers
# ---------------------------
output_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 5.xlsx'

# Create a new workbook; set the option to handle NaN/INF values.
workbook = xlsxwriter.Workbook(output_path, {'nan_inf_to_errors': True})
worksheet = workbook.add_worksheet("Ranking")

# Define the column headers.
columns = ["Timestamp", "RollingOLS Non-scaled 1 Rank", "Predicted_Return_Next_3M",
           "PERMNO", "Ticker", "Company", "Sector"]

current_row = 0

# Process each group sorted by Timestamp.
for timestamp, group_df in sorted(grouped_data, key=lambda x: x[0]):
    # Write the group header.
    header_text = f"Top 30 Stocks of {timestamp}"
    worksheet.write(current_row, 0, header_text)
    current_row += 1

    # Write the column headers.
    for col_idx, col_name in enumerate(columns):
        worksheet.write(current_row, col_idx, col_name)
    current_row += 1

    # Write the data rows.
    for _, row_data in group_df.iterrows():
        for col_idx, col_name in enumerate(columns):
            worksheet.write(current_row, col_idx, row_data[col_name])
        current_row += 1

    # Insert a blank row between groups.
    current_row += 1

workbook.close()
print(f"Ranking 5 successfully saved to {output_path}")


Ranking 5 successfully saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 5.xlsx


In [ ]:
# Install XlsxWriter if not already installed.
!pip install XlsxWriter

import pandas as pd
import xlsxwriter

# ---------------------------
# Step 1. Read the Final 25 File
# ---------------------------
input_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Final 25.xlsx'
df = pd.read_excel(input_path)

# ---------------------------
# Step 2. Exclude Rows from Specific Sectors
# ---------------------------
# Exclude rows where the Sector is Real Estate, Financials, or Healthcare.
df = df[~df['Sector'].isin(['Real Estate', 'Financials', 'Healthcare'])]

# ---------------------------
# Step 3. Create a Timestamp Column (YYYYMM)
# ---------------------------
# Assumes that the file contains 'Year' and 'Month' columns.
df['Timestamp'] = df['Year'].astype(str) + df['Month'].apply(lambda x: f"{int(x):02d}")

# ---------------------------
# Step 4. Remove Duplicate Entries
# ---------------------------
# For any duplicate PERMNO within the same Timestamp, keep the row with the highest Return_Next 3M.
df = df.sort_values('Return_Next 3M', ascending=False)
df = df.drop_duplicates(subset=['Timestamp', 'PERMNO'], keep='first')

# ---------------------------
# Step 5. Group by Timestamp, Rank Top 30 Winners, and Assign Actual Rank
# ---------------------------
grouped_data = []
for timestamp, group in df.groupby('Timestamp'):
    # Sort by Return_Next 3M in descending order and select the top 30 winners.
    group_sorted = group.sort_values(by='Return_Next 3M', ascending=False).head(30).copy()
    group_sorted = group_sorted.reset_index(drop=True)
    group_sorted['Actual Rank'] = range(1, len(group_sorted) + 1)

    # Select only the required columns in the specified order.
    group_sorted = group_sorted[['Timestamp', 'Actual Rank', 'Return_Next 3M', 'PERMNO', 'Ticker', 'Company', 'Sector']]

    grouped_data.append((timestamp, group_sorted))

# ---------------------------
# Step 6. Write the Output to "Ranking 6.xlsx" with Group Headers and a Total Row
# ---------------------------
output_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 6.xlsx'
workbook = xlsxwriter.Workbook(output_path, {'nan_inf_to_errors': True})
worksheet = workbook.add_worksheet("Ranking")

# Define the column headers.
columns = ["Timestamp", "Actual Rank", "Return_Next 3M", "PERMNO", "Ticker", "Company", "Sector"]

current_row = 0

# Process each group (sorted by Timestamp).
for timestamp, group_df in sorted(grouped_data, key=lambda x: x[0]):
    # Write a header row for the group.
    header_text = f"Top 30 Stocks of {timestamp}"
    worksheet.write(current_row, 0, header_text)
    current_row += 1

    # Write the column headers.
    for col_idx, col_name in enumerate(columns):
        worksheet.write(current_row, col_idx, col_name)
    current_row += 1

    # Write each data row for this Timestamp group.
    for _, row_data in group_df.iterrows():
        for col_idx, col_name in enumerate(columns):
            worksheet.write(current_row, col_idx, row_data[col_name])
        current_row += 1

    # Add a total row: sum the Return_Next 3M values for the group.
    total_return = group_df['Return_Next 3M'].sum()
    # Write "Total Return" in the Actual Rank column and the sum in the Return_Next 3M column.
    worksheet.write(current_row, 1, "Total Return")
    worksheet.write(current_row, 2, total_return)
    current_row += 1

    # Insert a blank row between groups for readability.
    current_row += 1

workbook.close()
print(f"Ranking 6 successfully saved to {output_path}")


<ipython-input-15-85e923573a5e>:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Timestamp'] = df['Year'].astype(str) + df['Month'].apply(lambda x: f"{int(x):02d}")


Ranking 6 successfully saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 6.xlsx


In [ ]:
# Install XlsxWriter if not already installed.
!pip install XlsxWriter

import pandas as pd
import xlsxwriter

# ---------------------------
# Step 1. Read the Final 34 File
# ---------------------------
final34_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Final 34.xlsx'
df34 = pd.read_excel(final34_path)

# ---------------------------
# Step 2. Exclude Rows Based on One‑Hot Encoded Dummy Variables
# ---------------------------
# Exclude rows where any of the following dummy columns equals 1.
df34 = df34[(df34['Sector_Real Estate'] != 1) &
            (df34['Sector_Financials'] != 1) &
            (df34['Sector_Healthcare'] != 1)]

# ---------------------------
# Step 3. Remove Duplicate Entries per Timestamp and PERMNO
# ---------------------------
# For rows with the same Timestamp and PERMNO, keep the one with the highest Predicted_Return_Next_3M.
df34 = df34.sort_values('Predicted_Return_Next_3M', ascending=False)
df34 = df34.drop_duplicates(subset=['Timestamp', 'PERMNO'], keep='first')

# ---------------------------
# Step 4. Merge in Company Info from the Company 1 File
# ---------------------------
company1_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Company 1.xlsx'
df_company = pd.read_excel(company1_path)
# Merge on PERMNO to bring in Ticker, Company, and Sector.
df34 = pd.merge(df34, df_company[['PERMNO', 'Ticker', 'Company', 'Sector']], on='PERMNO', how='left')

# ---------------------------
# Step 5. Group by Timestamp, Rank Top 30 Winners, and Assign Rank
# ---------------------------
# For each Timestamp, sort by Predicted_Return_Next_3M descending and select the top 30.
grouped_data = []
for timestamp, group in df34.groupby('Timestamp'):
    group_sorted = group.sort_values(by='Predicted_Return_Next_3M', ascending=False).head(30).copy()
    group_sorted = group_sorted.reset_index(drop=True)
    group_sorted['RollingOLS Non-scaled 1 Rank'] = range(1, len(group_sorted) + 1)

    # Select the required columns in the specified order.
    group_sorted = group_sorted[['Timestamp', 'RollingOLS Non-scaled 1 Rank',
                                   'Predicted_Return_Next_3M', 'PERMNO', 'Ticker', 'Company', 'Sector']]
    grouped_data.append((timestamp, group_sorted))

# ---------------------------
# Step 6. Write the Output to "Ranking 7.xlsx" with Group Headers and a Total Row
# ---------------------------
output_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 7.xlsx'
workbook = xlsxwriter.Workbook(output_path, {'nan_inf_to_errors': True})
worksheet = workbook.add_worksheet("Ranking")

# Define the column headers.
columns = ["Timestamp", "RollingOLS Non-scaled 1 Rank", "Predicted_Return_Next_3M", "PERMNO", "Ticker", "Company", "Sector"]

current_row = 0
# Process each group sorted by Timestamp.
for timestamp, group_df in sorted(grouped_data, key=lambda x: x[0]):
    # Write the group header.
    header_text = f"Top 30 Stocks of {timestamp}"
    worksheet.write(current_row, 0, header_text)
    current_row += 1

    # Write the column headers.
    for col_idx, col_name in enumerate(columns):
        worksheet.write(current_row, col_idx, col_name)
    current_row += 1

    # Write each data row for this group.
    for _, row_data in group_df.iterrows():
        for col_idx, col_name in enumerate(columns):
            worksheet.write(current_row, col_idx, row_data[col_name])
        current_row += 1

    # Add a total row: Sum the Predicted_Return_Next_3M values for this group.
    total_return = group_df['Predicted_Return_Next_3M'].sum()
    # Write "Total Return" in the "RollingOLS Non-scaled 1 Rank" column and the total value in the "Predicted_Return_Next_3M" column.
    worksheet.write(current_row, 1, "Total Return")
    worksheet.write(current_row, 2, total_return)
    current_row += 1

    # Insert a blank row between groups.
    current_row += 1

workbook.close()
print(f"Ranking 7 successfully saved to {output_path}")


Ranking 7 successfully saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 7.xlsx


In [ ]:
# Install required packages if not already installed.
!pip install XlsxWriter openpyxl

import pandas as pd
import xlsxwriter

def parse_ranking_file(filepath):
    """
    Parses a ranking file that uses group headers.
    Each group starts with a cell in Column A that reads "Top 30 Stocks of YYYYMM",
    followed by a row with column headers and then the data rows.
    Returns a dictionary mapping each Timestamp (YYYYMM) to a DataFrame containing that group's data.
    """
    # Read the entire file without a preset header.
    df = pd.read_excel(filepath, header=None, engine='openpyxl')
    groups = {}
    current_timestamp = None
    current_columns = None
    data_rows = []

    for _, row in df.iterrows():
        # Get the first cell as a string (if present).
        cell_val = str(row[0]) if pd.notnull(row[0]) else ""

        # Check if this row is a group header.
        if cell_val.startswith("Top 30 Stocks of "):
            # If we already were inside a group, save its data.
            if current_timestamp is not None and current_columns is not None and data_rows:
                groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
            # Start a new group by extracting the timestamp.
            current_timestamp = cell_val.split("Top 30 Stocks of ")[1].strip()
            current_columns = None
            data_rows = []
        # The row immediately following a group header is taken as the column headers.
        elif current_timestamp is not None and current_columns is None:
            current_columns = row.dropna().tolist()
        # Otherwise, if we are inside a group and the row is not entirely empty, treat it as a data row.
        elif current_timestamp is not None and current_columns is not None:
            if not row.isnull().all():
                # Only take as many cells as there are headers.
                row_data = row.tolist()[:len(current_columns)]
                data_rows.append(row_data)

    # Save the final group if available.
    if current_timestamp is not None and current_columns is not None and data_rows:
        groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)

    return groups

# ---------------------------
# Step 1. Read Both Ranking 6 and Ranking 7 Files
# ---------------------------
ranking6_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 6.xlsx'
ranking7_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 7.xlsx'

ranking6_groups = parse_ranking_file(ranking6_path)
ranking7_groups = parse_ranking_file(ranking7_path)

# ---------------------------
# Step 2. Combine the Data for Each Matching Timestamp Group
# ---------------------------
# We'll combine only for timestamps present in Ranking 6.
# For each such group, if a matching group exists in Ranking 7,
# we concatenate the two DataFrames horizontally (so that Ranking 7 data starts at column H, after 7 columns).
combined_groups = {}

for ts in ranking6_groups.keys():
    df6 = ranking6_groups[ts].reset_index(drop=True)
    if ts in ranking7_groups:
        df7 = ranking7_groups[ts].reset_index(drop=True)
        # Concatenate horizontally; this will place the Ranking 7 columns immediately after the Ranking 6 columns.
        combined_df = pd.concat([df6, df7], axis=1)
        combined_groups[ts] = combined_df
    else:
        # If there is no matching Ranking 7 group for this timestamp, use the Ranking 6 data as-is.
        combined_groups[ts] = df6

# ---------------------------
# Step 3. Write the Combined Data to "Ranking 8.xlsx"
# ---------------------------
output_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 8.xlsx'
workbook = xlsxwriter.Workbook(output_path, {'nan_inf_to_errors': True})
worksheet = workbook.add_worksheet("Combined Ranking")

current_row = 0

# Process groups in sorted order of timestamp.
for ts in sorted(combined_groups.keys()):
    # Write the group header in column A.
    header_text = f"Top 30 Stocks of {ts}"
    worksheet.write(current_row, 0, header_text)
    current_row += 1

    # Retrieve the combined DataFrame for this timestamp.
    combined_df = combined_groups[ts]

    # Write the column headers.
    for col_idx, col_name in enumerate(combined_df.columns):
        worksheet.write(current_row, col_idx, col_name)
    current_row += 1

    # Write the data rows.
    for _, row_data in combined_df.iterrows():
        for col_idx, cell in enumerate(row_data):
            worksheet.write(current_row, col_idx, cell)
        current_row += 1

    # Insert a blank row between groups.
    current_row += 1

workbook.close()
print(f"Combined data successfully saved to {output_path}")


Combined data successfully saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 8.xlsx


In [ ]:
# Install required packages if not already installed.
!pip install XlsxWriter openpyxl

import pandas as pd
import xlsxwriter

def parse_ranking_file(filepath):
    """
    Parses a ranking file that was written with group headers.
    Each group starts with a cell in column A containing "Top 30 Stocks of YYYYMM",
    followed by a row of column headers and then the data rows.

    Returns:
      A dictionary mapping each Timestamp (YYYYMM) to its corresponding DataFrame.
    """
    df = pd.read_excel(filepath, header=None, engine='openpyxl')
    groups = {}
    current_timestamp = None
    current_columns = None
    data_rows = []

    for _, row in df.iterrows():
        cell_val = str(row[0]) if pd.notnull(row[0]) else ""
        if cell_val.startswith("Top 30 Stocks of "):
            # If we were building a group, save it.
            if current_timestamp is not None and current_columns is not None and data_rows:
                groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
            # Start a new group: extract timestamp.
            current_timestamp = cell_val.split("Top 30 Stocks of ")[1].strip()
            current_columns = None
            data_rows = []
        elif current_timestamp is not None and current_columns is None:
            # The row immediately after the header is assumed to be column headers.
            current_columns = row.dropna().tolist()
        elif current_timestamp is not None and current_columns is not None:
            # If the row is not entirely empty, take the number of cells matching the headers.
            if not row.isnull().all():
                row_data = row.tolist()[:len(current_columns)]
                data_rows.append(row_data)
    # Save the final group if present.
    if current_timestamp is not None and current_columns is not None and data_rows:
        groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
    return groups

# Define file paths.
ranking6_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 6.xlsx'
ranking7_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 7.xlsx'

# ---------------------------
# Step 1. Parse Both Ranking Files
# ---------------------------
ranking6_groups = parse_ranking_file(ranking6_path)
ranking7_groups = parse_ranking_file(ranking7_path)

# ---------------------------
# Step 2. Combine Groups
# ---------------------------
# For each Timestamp present in Ranking 6, if Ranking 7 also has data for that Timestamp,
# horizontally concatenate the two DataFrames so that Ranking 7 data starts one column after Ranking 6's data.
combined_groups = {}
for ts in sorted(ranking6_groups.keys()):
    df6 = ranking6_groups[ts].reset_index(drop=True)
    if ts in ranking7_groups:
        df7 = ranking7_groups[ts].reset_index(drop=True)
        # Combine side by side.
        combined_df = pd.concat([df6, df7], axis=1)
    else:
        # If no matching group in Ranking 7, just use Ranking 6's data.
        combined_df = df6
    combined_groups[ts] = combined_df

# ---------------------------
# Step 3. Write the Combined Data to "Ranking 9.xlsx"
# ---------------------------
output_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 9.xlsx'
workbook = xlsxwriter.Workbook(output_path, {'nan_inf_to_errors': True})
worksheet = workbook.add_worksheet("Combined Ranking")

current_row = 0

# Loop over each group (Timestamp) sorted in ascending order.
for ts in sorted(combined_groups.keys()):
    # Write the group header in Column A.
    group_header = f"Top 30 Stocks of {ts}"
    worksheet.write(current_row, 0, group_header)
    current_row += 1

    combined_df = combined_groups[ts]
    ncols = combined_df.shape[1]

    # Write the column headers.
    for col_idx, col_name in enumerate(combined_df.columns):
        # Replace any header that equals the error string with an empty string.
        header_val = col_name if str(col_name).strip() != "=#NUM!" else ""
        worksheet.write(current_row, col_idx, header_val)
    current_row += 1

    # Write each row of data.
    for _, row_data in combined_df.iterrows():
        for col_idx, cell in enumerate(row_data):
            # If the cell value is a string that equals the error code, replace it with empty.
            if isinstance(cell, str) and cell.strip() == "=#NUM!":
                cell = ""
            worksheet.write(current_row, col_idx, cell)
        current_row += 1

    # Insert a blank row between groups.
    current_row += 1

workbook.close()
print(f"Combined data successfully saved to {output_path}")


Combined data successfully saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 9.xlsx


In [ ]:
# Install required packages if not already installed.
!pip install XlsxWriter openpyxl plotly

import pandas as pd
import xlsxwriter
import plotly.graph_objects as go

def parse_ranking_file(filepath):
    """
    Parses a ranking file that uses group headers.
    Each group starts with a cell in Column A that reads "Top 30 Stocks of YYYYMM",
    followed by a row with column headers and then the data rows.
    Returns a dictionary mapping each Timestamp (YYYYMM) to a DataFrame containing that group's data.
    """
    df = pd.read_excel(filepath, header=None, engine='openpyxl')
    groups = {}
    current_timestamp = None
    current_columns = None
    data_rows = []

    for _, row in df.iterrows():
        cell_val = str(row[0]) if pd.notnull(row[0]) else ""
        if cell_val.startswith("Top 30 Stocks of "):
            if current_timestamp is not None and current_columns is not None and data_rows:
                groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
            current_timestamp = cell_val.split("Top 30 Stocks of ")[1].strip()
            current_columns = None
            data_rows = []
        elif current_timestamp is not None and current_columns is None:
            current_columns = row.dropna().tolist()
        elif current_timestamp is not None and current_columns is not None:
            if not row.isnull().all():
                row_data = row.tolist()[:len(current_columns)]
                data_rows.append(row_data)
    if current_timestamp is not None and current_columns is not None and data_rows:
        groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
    return groups

# ---------------------------
# Step 1. Read Ranking 8 and parse its groups.
# ---------------------------
ranking8_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 8.xlsx'
ranking8_groups = parse_ranking_file(ranking8_path)

# ---------------------------
# Step 2. Extract Total Return values for each Timestamp.
# ---------------------------
total_data = []
for ts, df_group in ranking8_groups.items():
    actual_return = None
    predicted_return = None
    if df_group.shape[1] >= 3:
        mask_actual = df_group.iloc[:, 1] == "Total Return"
        if mask_actual.any():
            actual_return = df_group.loc[mask_actual].iloc[0, 2]
    if df_group.shape[1] >= 9:
        mask_pred = df_group.iloc[:, 8] == "Total Return"
        if mask_pred.any():
            predicted_return = df_group.loc[mask_pred].iloc[0, 9]
    total_data.append({
        "Timestamp": ts,
        "Actual Portfolio Return": actual_return,
        "RollingOLS Non-scaled 1 Return": predicted_return
    })

df_total = pd.DataFrame(total_data)
df_total = df_total.sort_values("Timestamp")

# ---------------------------
# Step 3. Create an Extremely Fast Animated Chart using Plotly.
# ---------------------------
timestamps = df_total["Timestamp"].tolist()
actual_returns = df_total["Actual Portfolio Return"].tolist()
predicted_returns = df_total["RollingOLS Non-scaled 1 Return"].tolist()

x_vals = list(range(len(timestamps)))

# Build frames: each frame reveals data up to that timestamp.
frames = []
for i in range(len(df_total)):
    frames.append(go.Frame(
        data=[
            go.Scatter(x=x_vals[:i+1], y=actual_returns[:i+1],
                       mode='lines+markers', name='Actual Portfolio Return'),
            go.Scatter(x=x_vals[:i+1], y=predicted_returns[:i+1],
                       mode='lines+markers', name='RollingOLS Non-scaled 1 Return')
        ],
        name=str(i)
    ))

# Define the initial data (first point).
init_data = [
    go.Scatter(x=[x_vals[0]], y=[actual_returns[0]],
               mode='lines+markers', name='Actual Portfolio Return'),
    go.Scatter(x=[x_vals[0]], y=[predicted_returns[0]],
               mode='lines+markers', name='RollingOLS Non-scaled 1 Return')
]

fig = go.Figure(
    data=init_data,
    layout=go.Layout(
        title="Animated Portfolio Returns",
        xaxis=dict(
            title="Timestamp",
            tickmode="array",
            tickvals=x_vals,
            ticktext=timestamps
        ),
        yaxis=dict(title="Return"),
        updatemenus=[
            dict(
                type="buttons",
                buttons=[
                    dict(label="Play",
                         method="animate",
                         args=[None, {"frame": {"duration": 50, "redraw": True},
                                      "transition": {"duration": 0},
                                      "fromcurrent": True}]),
                    dict(label="Pause",
                         method="animate",
                         args=[[None], {"frame": {"duration": 0, "redraw": False},
                                        "mode": "immediate",
                                        "transition": {"duration": 0}}])
                ]
            )
        ]
    ),
    frames=frames
)

fig.show()


In [ ]:
# Install required packages if not already installed.
!pip install XlsxWriter openpyxl plotly

import pandas as pd
import plotly.graph_objects as go

def parse_ranking_file(filepath):
    """
    Parses a ranking file that uses group headers.
    Each group starts with a cell in Column A that reads "Top 30 Stocks of YYYYMM",
    followed by a row with column headers and then the data rows.
    Returns a dictionary mapping each Timestamp (YYYYMM) to a DataFrame containing that group's data.
    """
    df = pd.read_excel(filepath, header=None, engine='openpyxl')
    groups = {}
    current_timestamp = None
    current_columns = None
    data_rows = []

    for _, row in df.iterrows():
        cell_val = str(row[0]) if pd.notnull(row[0]) else ""
        if cell_val.startswith("Top 30 Stocks of "):
            if current_timestamp is not None and current_columns is not None and data_rows:
                groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
            current_timestamp = cell_val.split("Top 30 Stocks of ")[1].strip()
            current_columns = None
            data_rows = []
        elif current_timestamp is not None and current_columns is None:
            current_columns = row.dropna().tolist()
        elif current_timestamp is not None and current_columns is not None:
            if not row.isnull().all():
                row_data = row.tolist()[:len(current_columns)]
                data_rows.append(row_data)
    if current_timestamp is not None and current_columns is not None and data_rows:
        groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
    return groups

# ---------------------------
# Step 1. Read Ranking 8 and parse its groups.
# ---------------------------
ranking8_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 8.xlsx'
ranking8_groups = parse_ranking_file(ranking8_path)

# ---------------------------
# Step 2. Extract Total Return values for each Timestamp.
# ---------------------------
# For Ranking 6 side:
#   Expected columns: ["Timestamp", "Actual Rank", "Return_Next 3M", "PERMNO", "Ticker", "Company", "Sector"]
#   The "Total Return" row is identified by "Total Return" in the "Actual Rank" column (index 1)
#   and its value is in the "Return_Next 3M" column (index 2).
# For Ranking 7 side:
#   These columns are appended (columns 7 to 13) with expected header row:
#   ["Timestamp", "RollingOLS Non-scaled 1 Rank", "Predicted_Return_Next_3M", "PERMNO", "Ticker", "Company", "Sector"]
#   with the total row having "Total Return" in the "RollingOLS Non-scaled 1 Rank" column (index 8)
#   and the sum in the "Predicted_Return_Next_3M" column (index 9).
total_data = []
for ts, df_group in ranking8_groups.items():
    actual_return = None
    predicted_return = None
    if df_group.shape[1] >= 3:
        mask_actual = df_group.iloc[:, 1] == "Total Return"
        if mask_actual.any():
            actual_return = df_group.loc[mask_actual].iloc[0, 2]
    if df_group.shape[1] >= 9:
        mask_pred = df_group.iloc[:, 8] == "Total Return"
        if mask_pred.any():
            predicted_return = df_group.loc[mask_pred].iloc[0, 9]
    total_data.append({
        "Timestamp": ts,
        "Actual Portfolio Return": actual_return,
        "RollingOLS Non-scaled 1 Return": predicted_return
    })

df_total = pd.DataFrame(total_data)
df_total = df_total.sort_values("Timestamp")  # Ensure timestamps are sorted

# ---------------------------
# Step 3. Create a Fixed (Non-Animated) Chart using Plotly.
# ---------------------------
fig = go.Figure(
    data=[
        go.Scatter(x=df_total["Timestamp"], y=df_total["Actual Portfolio Return"],
                   mode='lines+markers', name='Actual Portfolio Return'),
        go.Scatter(x=df_total["Timestamp"], y=df_total["RollingOLS Non-scaled 1 Return"],
                   mode='lines+markers', name='RollingOLS Non-scaled 1 Return')
    ],
    layout=go.Layout(
        title="Portfolio Returns by Timestamp",
        xaxis=dict(title="Timestamp"),
        yaxis=dict(title="Return")
    )
)

fig.show()


In [ ]:
# Install XlsxWriter if not already installed.
!pip install XlsxWriter

import pandas as pd
import xlsxwriter

# ---------------------------
# Step 1. Read the Final 47 File
# ---------------------------
final47_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Final 47.xlsx'
df47 = pd.read_excel(final47_path)

# If there's no Timestamp column but Year and Month exist, create one.
if 'Timestamp' not in df47.columns and 'Year' in df47.columns and 'Month' in df47.columns:
    df47['Timestamp'] = df47['Year'].astype(str) + df47['Month'].apply(lambda x: f"{int(x):02d}")

# ---------------------------
# Step 2. Remove Duplicate Entries
# ---------------------------
# For each Timestamp and PERMNO, keep the row with the highest Predicted_Return_Next_3M.
df47 = df47.sort_values('Predicted_Return_Next_3M', ascending=False)
df47_unique = df47.drop_duplicates(subset=['Timestamp', 'PERMNO'], keep='first')

# ---------------------------
# Step 3. Merge in Company Info from Company 1 File
# ---------------------------
company1_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Company 1.xlsx'
df_company = pd.read_excel(company1_path)
df47_merged = pd.merge(df47_unique,
                       df_company[['PERMNO', 'Ticker', 'Company', 'Sector']],
                       on='PERMNO',
                       how='left')

# ---------------------------
# Step 4. Group by Timestamp and Rank Top 30 Winners
# ---------------------------
# For each Timestamp group, sort by Predicted_Return_Next_3M (descending) and select the top 30 rows.
grouped_data = []
for ts, group in df47_merged.groupby('Timestamp'):
    group_sorted = group.sort_values(by='Predicted_Return_Next_3M', ascending=False).head(30).copy()
    group_sorted = group_sorted.reset_index(drop=True)
    group_sorted['RollingOLS Single-scaled 2 Rank'] = range(1, len(group_sorted) + 1)

    # Select only the required columns in the specified order.
    group_sorted = group_sorted[['Timestamp', 'RollingOLS Single-scaled 2 Rank',
                                   'Predicted_Return_Next_3M', 'PERMNO', 'Ticker', 'Company', 'Sector']]
    grouped_data.append((ts, group_sorted))

# ---------------------------
# Step 5. Write the Output to "Ranking 10.xlsx" with Group Headers and a Total Row
# ---------------------------
output_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 10.xlsx'
workbook = xlsxwriter.Workbook(output_path, {'nan_inf_to_errors': True})
worksheet = workbook.add_worksheet("Ranking")

# Define the column headers.
columns = ["Timestamp", "RollingOLS Single-scaled 2 Rank", "Predicted_Return_Next_3M",
           "PERMNO", "Ticker", "Company", "Sector"]

current_row = 0

# Process each group sorted by Timestamp.
for ts, group_df in sorted(grouped_data, key=lambda x: x[0]):
    # Write the group header.
    header_text = f"Top 30 Stocks of {ts}"
    worksheet.write(current_row, 0, header_text)
    current_row += 1

    # Write the column headers.
    for col_idx, col_name in enumerate(columns):
        worksheet.write(current_row, col_idx, col_name)
    current_row += 1

    # Write each row of data for this Timestamp group.
    for _, row_data in group_df.iterrows():
        for col_idx, col_name in enumerate(columns):
            worksheet.write(current_row, col_idx, row_data[col_name])
        current_row += 1

    # Add a total row: sum the Predicted_Return_Next_3M values for the group.
    total_return = group_df['Predicted_Return_Next_3M'].sum()
    worksheet.write(current_row, 1, "Total Return")
    worksheet.write(current_row, 2, total_return)
    current_row += 1

    # Insert a blank row between groups for readability.
    current_row += 1

workbook.close()
print(f"Ranking 10 successfully saved to {output_path}")


Ranking 10 successfully saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 10.xlsx


In [ ]:
# Install XlsxWriter if not already installed.
!pip install XlsxWriter

import pandas as pd
import xlsxwriter

# ---------------------------
# Step 1. Read the Final 46 File
# ---------------------------
final46_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Final 46.xlsx'
df46 = pd.read_excel(final46_path)

# If a Timestamp column doesn't exist but Year and Month do, create one.
if 'Timestamp' not in df46.columns and 'Year' in df46.columns and 'Month' in df46.columns:
    df46['Timestamp'] = df46['Year'].astype(str) + df46['Month'].apply(lambda x: f"{int(x):02d}")

# ---------------------------
# Step 2. Remove Duplicate Entries
# ---------------------------
# Sort by Predicted_Return_Next_3M descending and drop duplicates by Timestamp and PERMNO.
df46 = df46.sort_values('Predicted_Return_Next_3M', ascending=False)
df46_unique = df46.drop_duplicates(subset=['Timestamp', 'PERMNO'], keep='first')

# ---------------------------
# Step 3. Merge in Company Info from Company 1 File
# ---------------------------
company1_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Company 1.xlsx'
df_company = pd.read_excel(company1_path)
df46_merged = pd.merge(df46_unique,
                       df_company[['PERMNO', 'Ticker', 'Company', 'Sector']],
                       on='PERMNO',
                       how='left')

# ---------------------------
# Step 4. Group by Timestamp and Rank Top 30 Winners
# ---------------------------
grouped_data = []
for ts, group in df46_merged.groupby('Timestamp'):
    # Sort by Predicted_Return_Next_3M descending and take top 30.
    group_sorted = group.sort_values(by='Predicted_Return_Next_3M', ascending=False).head(30).copy()
    group_sorted = group_sorted.reset_index(drop=True)
    group_sorted['RollingOLS Double-scaled Rank'] = range(1, len(group_sorted) + 1)

    # Select only the required columns in the specified order.
    group_sorted = group_sorted[['Timestamp', 'RollingOLS Double-scaled Rank',
                                   'Predicted_Return_Next_3M', 'PERMNO', 'Ticker', 'Company', 'Sector']]
    grouped_data.append((ts, group_sorted))

# ---------------------------
# Step 5. Write the Output to "Ranking 11.xlsx" with Group Headers and Total Row
# ---------------------------
output_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 11.xlsx'
workbook = xlsxwriter.Workbook(output_path, {'nan_inf_to_errors': True})
worksheet = workbook.add_worksheet("Ranking")

# Define the column headers.
columns = ["Timestamp", "RollingOLS Double-scaled Rank", "Predicted_Return_Next_3M", "PERMNO", "Ticker", "Company", "Sector"]

current_row = 0

# Process each group sorted by Timestamp.
for ts, group_df in sorted(grouped_data, key=lambda x: x[0]):
    # Write a header row for this group.
    header_text = f"Top 30 Stocks of {ts}"
    worksheet.write(current_row, 0, header_text)
    current_row += 1

    # Write the column headers.
    for col_idx, col_name in enumerate(columns):
        worksheet.write(current_row, col_idx, col_name)
    current_row += 1

    # Write each row of data for this Timestamp group.
    for _, row_data in group_df.iterrows():
        for col_idx, col_name in enumerate(columns):
            worksheet.write(current_row, col_idx, row_data[col_name])
        current_row += 1

    # Add a total row: Sum the Predicted_Return_Next_3M values for this group.
    total_return = group_df['Predicted_Return_Next_3M'].sum()
    worksheet.write(current_row, 1, "Total Return")
    worksheet.write(current_row, 2, total_return)
    current_row += 1

    # Insert a blank row between groups for readability.
    current_row += 1

workbook.close()
print(f"Ranking 11 successfully saved to {output_path}")


Ranking 11 successfully saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 11.xlsx


In [ ]:
# Install XlsxWriter if not already installed.
!pip install XlsxWriter

import pandas as pd
import xlsxwriter

# ---------------------------
# Step 1. Read the Final 48 File
# ---------------------------
final48_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Final 48.xlsx'
df48 = pd.read_excel(final48_path)

# If there's no Timestamp column but Year and Month exist, create one.
if 'Timestamp' not in df48.columns and 'Year' in df48.columns and 'Month' in df48.columns:
    df48['Timestamp'] = df48['Year'].astype(str) + df48['Month'].apply(lambda x: f"{int(x):02d}")

# ---------------------------
# Step 2. Remove Duplicate Entries
# ---------------------------
# Sort by predicted_prob (highest first) and drop duplicates by Timestamp and PERMNO.
df48 = df48.sort_values('predicted_prob', ascending=False)
df48_unique = df48.drop_duplicates(subset=['Timestamp', 'PERMNO'], keep='first')

# ---------------------------
# Step 3. Merge in Company Info from Company 1 File
# ---------------------------
company1_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Company 1.xlsx'
df_company = pd.read_excel(company1_path)
df48_merged = pd.merge(df48_unique,
                       df_company[['PERMNO', 'Ticker', 'Company', 'Sector']],
                       on='PERMNO',
                       how='left')

# ---------------------------
# Step 4. Group by Timestamp and Rank Top 30 Winners
# ---------------------------
# For each Timestamp, sort by predicted_prob (descending) and select the top 30 winners.
grouped_data = []
for ts, group in df48_merged.groupby('Timestamp'):
    group_sorted = group.sort_values(by='predicted_prob', ascending=False).head(30).copy()
    group_sorted = group_sorted.reset_index(drop=True)
    group_sorted['Logistic Single-scaled Rank'] = range(1, len(group_sorted) + 1)
    # Keep only the required columns.
    group_sorted = group_sorted[['Timestamp', 'Logistic Single-scaled Rank',
                                   'predicted_prob', 'PERMNO', 'Ticker', 'Company', 'Sector']]
    grouped_data.append((ts, group_sorted))

# ---------------------------
# Step 5. Write the Output to "Ranking 12.xlsx" with Group Headers and Total Row
# ---------------------------
output_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 12.xlsx'
workbook = xlsxwriter.Workbook(output_path, {'nan_inf_to_errors': True})
worksheet = workbook.add_worksheet("Ranking")

# Define the output column headers.
columns = ["Timestamp", "Logistic Single-scaled Rank", "predicted_prob", "PERMNO", "Ticker", "Company", "Sector"]

current_row = 0

# Process each group (sorted by Timestamp).
for ts, group_df in sorted(grouped_data, key=lambda x: x[0]):
    # Write a header row for the group.
    header_text = f"Top 30 Stocks of {ts}"
    worksheet.write(current_row, 0, header_text)
    current_row += 1

    # Write the column headers.
    for col_idx, col_name in enumerate(columns):
        worksheet.write(current_row, col_idx, col_name)
    current_row += 1

    # Write each data row for this Timestamp group.
    for _, row_data in group_df.iterrows():
        for col_idx, col_name in enumerate(columns):
            worksheet.write(current_row, col_idx, row_data[col_name])
        current_row += 1

    # Compute the total of the Return_Next 3M values for the top 30 winners.
    # (Assumes the Final 48 file contains a 'Return_Next 3M' column.)
    total_return = df48_merged[
        (df48_merged['Timestamp'] == ts) &
        (df48_merged['PERMNO'].isin(group_df['PERMNO']))
    ]['Return_Next 3M'].sum()

    # Write a total row:
    # Write "Total Return" in the "Logistic Single-scaled Rank" column (index 1)
    # and the total_return in the 'predicted_prob' column (index 2).
    worksheet.write(current_row, 1, "Total Return")
    worksheet.write(current_row, 2, total_return)
    current_row += 1

    # Insert a blank row between groups.
    current_row += 1

workbook.close()
print(f"Ranking 12 successfully saved to {output_path}")


Ranking 12 successfully saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 12.xlsx


In [ ]:
# Install XlsxWriter if not already installed.
!pip install XlsxWriter

import pandas as pd
import xlsxwriter

# ---------------------------
# Step 1. Read the Final 48 File
# ---------------------------
final48_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Final 48.xlsx'
df48 = pd.read_excel(final48_path)

# Create a Timestamp column if not present (assuming Year and Month columns exist)
if 'Timestamp' not in df48.columns and 'Year' in df48.columns and 'Month' in df48.columns:
    df48['Timestamp'] = df48['Year'].astype(str) + df48['Month'].apply(lambda x: f"{int(x):02d}")

# ---------------------------
# Step 2. Remove Duplicate Entries
# ---------------------------
# Sort by predicted_prob descending and drop duplicates for each (Timestamp, PERMNO)
df48 = df48.sort_values('predicted_prob', ascending=False)
df48_unique = df48.drop_duplicates(subset=['Timestamp', 'PERMNO'], keep='first')

# ---------------------------
# Step 3. Merge Company Data from Company 1 File
# ---------------------------
company1_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Company 1.xlsx'
df_company = pd.read_excel(company1_path)
df48_merged = pd.merge(df48_unique,
                       df_company[['PERMNO', 'Ticker', 'Company', 'Sector']],
                       on='PERMNO',
                       how='left')

# ---------------------------
# Step 4. Group by Timestamp and Rank Top 30 Winners
# ---------------------------
grouped_data = []

# Process each Timestamp group
for ts, group in df48_merged.groupby('Timestamp'):
    # Sort by predicted_prob descending and take the top 30 rows.
    group_sorted = group.sort_values(by='predicted_prob', ascending=False).head(30).copy()
    group_sorted = group_sorted.reset_index(drop=True)
    # Assign rank numbers 1 to N (where N ≤ 30)
    group_sorted['Logistic Single-scaled Rank'] = range(1, len(group_sorted) + 1)

    # Retain only the required columns.
    group_sorted = group_sorted[['Timestamp', 'Logistic Single-scaled Rank', 'predicted_prob', 'PERMNO', 'Ticker', 'Company', 'Sector']]

    # Append a total row that sums the predicted_prob values for the group.
    total_return = group_sorted['predicted_prob'].astype(float).sum()
    total_row = {
        'Timestamp': ts,
        'Logistic Single-scaled Rank': "Total Return",
        'predicted_prob': total_return,
        'PERMNO': "",
        'Ticker': "",
        'Company': "",
        'Sector': ""
    }
    group_sorted = pd.concat([group_sorted, pd.DataFrame([total_row])], ignore_index=True)

    grouped_data.append((ts, group_sorted))

# ---------------------------
# Step 5. Write the Output to "Ranking 12.xlsx"
# ---------------------------
output_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 12.xlsx'
workbook = xlsxwriter.Workbook(output_path, {'nan_inf_to_errors': True})
worksheet = workbook.add_worksheet("Ranking")

# Define the output column headers.
columns = ["Timestamp", "Logistic Single-scaled Rank", "predicted_prob", "PERMNO", "Ticker", "Company", "Sector"]

current_row = 0
# Write each group in sorted order by Timestamp.
for ts, group_df in sorted(grouped_data, key=lambda x: x[0]):
    # Write the group header.
    header_text = f"Top 30 Stocks of {ts}"
    worksheet.write(current_row, 0, header_text)
    current_row += 1

    # Write the column headers.
    for col_idx, col_name in enumerate(columns):
        worksheet.write(current_row, col_idx, col_name)
    current_row += 1

    # Write each row of data for this group.
    for _, row_data in group_df.iterrows():
        for col_idx, col_name in enumerate(columns):
            worksheet.write(current_row, col_idx, row_data[col_name])
        current_row += 1

    # Insert a blank row between groups.
    current_row += 1

workbook.close()
print(f"Ranking 12 successfully saved to {output_path}")


Ranking 12 successfully saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 12.xlsx


In [ ]:
# Install XlsxWriter if not already installed.
!pip install XlsxWriter

import pandas as pd
import xlsxwriter

# ---------------------------
# Step 1. Read the Final 49 File
# ---------------------------
final49_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Final 49.xlsx'
df49 = pd.read_excel(final49_path)

# If there is no Timestamp column but Year and Month exist, create one.
if 'Timestamp' not in df49.columns and 'Year' in df49.columns and 'Month' in df49.columns:
    df49['Timestamp'] = df49['Year'].astype(str) + df49['Month'].apply(lambda x: f"{int(x):02d}")

# ---------------------------
# Step 2. Remove Duplicate Entries
# ---------------------------
# Sort by predicted_prob in descending order and drop duplicates for each Timestamp and PERMNO.
df49 = df49.sort_values('predicted_prob', ascending=False)
df49_unique = df49.drop_duplicates(subset=['Timestamp', 'PERMNO'], keep='first')

# ---------------------------
# Step 3. Merge in Company Info from Company 1 File
# ---------------------------
company1_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Company 1.xlsx'
df_company = pd.read_excel(company1_path)
df49_merged = pd.merge(df49_unique,
                       df_company[['PERMNO', 'Ticker', 'Company', 'Sector']],
                       on='PERMNO',
                       how='left')

# ---------------------------
# Step 4. Group by Timestamp and Rank Top 30 Winners
# ---------------------------
# For each Timestamp, sort by predicted_prob (descending) and select the top 30.
grouped_data = []
for ts, group in df49_merged.groupby('Timestamp'):
    group_sorted = group.sort_values(by='predicted_prob', ascending=False).head(30).copy()
    group_sorted = group_sorted.reset_index(drop=True)
    group_sorted['Logistic Double-scaled Rank'] = range(1, len(group_sorted) + 1)

    # Keep only the required columns in the specified order.
    group_sorted = group_sorted[['Timestamp', 'Logistic Double-scaled Rank', 'predicted_prob',
                                   'PERMNO', 'Ticker', 'Company', 'Sector']]
    grouped_data.append((ts, group_sorted))

# ---------------------------
# Step 5. Write the Output to "Ranking 13.xlsx" with Group Headers and a Total Row
# ---------------------------
output_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 13.xlsx'
workbook = xlsxwriter.Workbook(output_path, {'nan_inf_to_errors': True})
worksheet = workbook.add_worksheet("Ranking")

# Define the column headers.
columns = ["Timestamp", "Logistic Double-scaled Rank", "predicted_prob", "PERMNO", "Ticker", "Company", "Sector"]

current_row = 0

# Process each group sorted by Timestamp.
for ts, group_df in sorted(grouped_data, key=lambda x: x[0]):
    # Write the header row for the group.
    header_text = f"Top 30 Stocks of {ts}"
    worksheet.write(current_row, 0, header_text)
    current_row += 1

    # Write the column headers.
    for col_idx, col_name in enumerate(columns):
        worksheet.write(current_row, col_idx, col_name)
    current_row += 1

    # Write each data row for this Timestamp group.
    for _, row_data in group_df.iterrows():
        for col_idx, col_name in enumerate(columns):
            worksheet.write(current_row, col_idx, row_data[col_name])
        current_row += 1

    # Add a total row: sum the Return_Next 3M values for the top 30 winners.
    # (Assumes the Final 49 file contains a 'Return_Next 3M' column.)
    total_return = df49_merged[
        (df49_merged['Timestamp'] == ts) &
        (df49_merged['PERMNO'].isin(group_df['PERMNO']))
    ]['Return_Next 3M'].sum()

    # Write the total row: "Total Return" in the "Logistic Double-scaled Rank" column (index 1)
    # and the total sum in the predicted_prob column (index 2).
    worksheet.write(current_row, 1, "Total Return")
    worksheet.write(current_row, 2, total_return)
    current_row += 1

    # Insert a blank row between groups for readability.
    current_row += 1

workbook.close()
print(f"Ranking 13 successfully saved to {output_path}")


Ranking 13 successfully saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 13.xlsx


In [ ]:
# Install XlsxWriter if not already installed.
!pip install XlsxWriter

import pandas as pd
import xlsxwriter

# ---------------------------
# Step 1. Read the Final 49 File
# ---------------------------
final49_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Final 49.xlsx'
df49 = pd.read_excel(final49_path)

# Create a Timestamp column if not present (assuming Year and Month columns exist)
if 'Timestamp' not in df49.columns and 'Year' in df49.columns and 'Month' in df49.columns:
    df49['Timestamp'] = df49['Year'].astype(str) + df49['Month'].apply(lambda x: f"{int(x):02d}")

# ---------------------------
# Step 2. Remove Duplicate Entries
# ---------------------------
# Sort by predicted_prob (highest first) and drop duplicates based on (Timestamp, PERMNO)
df49 = df49.sort_values('predicted_prob', ascending=False)
df49_unique = df49.drop_duplicates(subset=['Timestamp', 'PERMNO'], keep='first')

# ---------------------------
# Step 3. Merge in Company Data from Company 1 File
# ---------------------------
company1_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Company 1.xlsx'
df_company = pd.read_excel(company1_path)
df49_merged = pd.merge(df49_unique,
                       df_company[['PERMNO', 'Ticker', 'Company', 'Sector']],
                       on='PERMNO',
                       how='left')

# ---------------------------
# Step 4. Group by Timestamp and Rank Top 30 Winners
# ---------------------------
grouped_data = []
for ts, group in df49_merged.groupby('Timestamp'):
    # For each Timestamp, sort by predicted_prob descending and take top 30 rows.
    group_sorted = group.sort_values(by='predicted_prob', ascending=False).head(30).copy()
    group_sorted = group_sorted.reset_index(drop=True)
    group_sorted['Logistic Double-scaled Rank'] = range(1, len(group_sorted) + 1)

    # Retain only the required columns.
    group_sorted = group_sorted[['Timestamp', 'Logistic Double-scaled Rank', 'predicted_prob', 'PERMNO', 'Ticker', 'Company', 'Sector']]

    # Append a total row that sums the predicted_prob values for the group.
    total_val = group_sorted['predicted_prob'].astype(float).sum()
    total_row = {
        'Timestamp': ts,
        'Logistic Double-scaled Rank': "Total Return",
        'predicted_prob': total_val,
        'PERMNO': "",
        'Ticker': "",
        'Company': "",
        'Sector': ""
    }
    group_sorted = pd.concat([group_sorted, pd.DataFrame([total_row])], ignore_index=True)

    grouped_data.append((ts, group_sorted))

# ---------------------------
# Step 5. Write the Output to "Ranking 13.xlsx"
# ---------------------------
output_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 13.xlsx'
workbook = xlsxwriter.Workbook(output_path, {'nan_inf_to_errors': True})
worksheet = workbook.add_worksheet("Ranking")

# Define the output column headers.
columns = ["Timestamp", "Logistic Double-scaled Rank", "predicted_prob", "PERMNO", "Ticker", "Company", "Sector"]

current_row = 0
# Process groups in sorted order by Timestamp.
for ts, group_df in sorted(grouped_data, key=lambda x: x[0]):
    # Write the group header.
    header_text = f"Top 30 Stocks of {ts}"
    worksheet.write(current_row, 0, header_text)
    current_row += 1

    # Write the column headers.
    for col_idx, col_name in enumerate(columns):
        worksheet.write(current_row, col_idx, col_name)
    current_row += 1

    # Write each data row for this group.
    for _, row_data in group_df.iterrows():
        for col_idx, col_name in enumerate(columns):
            worksheet.write(current_row, col_idx, row_data[col_name])
        current_row += 1

    # Insert a blank row between groups.
    current_row += 1

workbook.close()
print(f"Ranking 13 successfully saved to {output_path}")


Ranking 13 successfully saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 13.xlsx


In [ ]:
# Install required packages if not already installed.
!pip install XlsxWriter openpyxl

import pandas as pd
import xlsxwriter

# ---------------------------
# Helper: Parse Ranking File with Group Headers
# ---------------------------
def parse_ranking_file(filepath):
    """
    Parses a ranking file that uses group headers.
    Each group starts with a cell in Column A that reads "Top 30 Stocks of YYYYMM",
    followed by a row with column headers and then the data rows.
    Returns a dictionary mapping each Timestamp (YYYYMM) to a DataFrame containing that group's data.
    """
    df = pd.read_excel(filepath, header=None, engine='openpyxl')
    groups = {}
    current_timestamp = None
    current_columns = None
    data_rows = []

    for _, row in df.iterrows():
        cell_val = str(row[0]) if pd.notnull(row[0]) else ""
        if cell_val.startswith("Top 30 Stocks of "):
            if current_timestamp is not None and current_columns is not None and data_rows:
                groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
            current_timestamp = cell_val.split("Top 30 Stocks of ")[1].strip()
            current_columns = None
            data_rows = []
        elif current_timestamp is not None and current_columns is None:
            current_columns = row.dropna().tolist()
        elif current_timestamp is not None and current_columns is not None:
            if not row.isnull().all():
                row_data = row.tolist()[:len(current_columns)]
                data_rows.append(row_data)
    if current_timestamp is not None and current_columns is not None and data_rows:
        groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
    return groups

# ---------------------------
# Step 1. Read the Ranking 7 File
# ---------------------------
ranking7_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 7.xlsx'
ranking7_groups = parse_ranking_file(ranking7_path)

# ---------------------------
# Step 2. Read the Final 25 File and Build a Lookup Dictionary
# ---------------------------
final25_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Final 25.xlsx'
df25 = pd.read_excel(final25_path)

# If Timestamp column is missing and Year/Month exist, create one.
if 'Timestamp' not in df25.columns and 'Year' in df25.columns and 'Month' in df25.columns:
    df25['Timestamp'] = df25['Year'].astype(str) + df25['Month'].apply(lambda x: f"{int(x):02d}")

# Sort by Return_Next 3M descending and drop duplicates (keeping highest value) for each (Timestamp, PERMNO).
df25 = df25.sort_values('Return_Next 3M', ascending=False)
df25_unique = df25.drop_duplicates(subset=['Timestamp', 'PERMNO'], keep='first')

# Build a dictionary mapping (Timestamp, PERMNO) -> Return_Next 3M
final25_lookup = {(row['Timestamp'], row['PERMNO']): row['Return_Next 3M']
                  for _, row in df25_unique.iterrows()}

# ---------------------------
# Step 3. Update Ranking 7 Groups with Final 25's Return_Next 3M Values
# ---------------------------
# Ranking 7 columns are expected to be:
# ["Timestamp", "RollingOLS Non-scaled 1 Rank", "Predicted_Return_Next_3M", "PERMNO", "Ticker", "Company", "Sector"]
for ts, group_df in ranking7_groups.items():
    # Iterate over rows that are not the total row.
    for idx, row in group_df.iterrows():
        if row["RollingOLS Non-scaled 1 Rank"] != "Total Return":
            key = (row["Timestamp"], row["PERMNO"])
            if key in final25_lookup:
                # Replace Predicted_Return_Next_3M with the matching Return_Next 3M value.
                group_df.at[idx, "Predicted_Return_Next_3M"] = final25_lookup[key]
    # Recalculate total: sum of updated "Predicted_Return_Next_3M" for data rows (exclude total row).
    numeric_rows = group_df[group_df["RollingOLS Non-scaled 1 Rank"] != "Total Return"]
    # Ensure conversion to float (in case values are strings)
    total_val = numeric_rows["Predicted_Return_Next_3M"].astype(float).sum()
    # Update the total row (if exists) – it's the row where "RollingOLS Non-scaled 1 Rank" equals "Total Return"
    total_row = group_df[group_df["RollingOLS Non-scaled 1 Rank"] == "Total Return"]
    if not total_row.empty:
        total_idx = total_row.index[0]
        group_df.at[total_idx, "Predicted_Return_Next_3M"] = total_val
    # Update the group in the dictionary.
    ranking7_groups[ts] = group_df

# ---------------------------
# Step 4. Write the Updated Groups to "Ranking 14.xlsx"
# ---------------------------
output_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 14.xlsx'
workbook = xlsxwriter.Workbook(output_path, {'nan_inf_to_errors': True})
worksheet = workbook.add_worksheet("Ranking")

# The columns remain as in Ranking 7:
columns = ["Timestamp", "RollingOLS Non-scaled 1 Rank", "Predicted_Return_Next_3M",
           "PERMNO", "Ticker", "Company", "Sector"]

current_row = 0
# Process groups in sorted order by Timestamp.
for ts, group_df in sorted(ranking7_groups.items(), key=lambda x: x[0]):
    # Write a header row for the group.
    header_text = f"Top 30 Stocks of {ts}"
    worksheet.write(current_row, 0, header_text)
    current_row += 1

    # Write the column headers.
    for col_idx, col_name in enumerate(columns):
        worksheet.write(current_row, col_idx, col_name)
    current_row += 1

    # Write the data rows for this group.
    for _, row_data in group_df.iterrows():
        for col_idx, col_name in enumerate(columns):
            worksheet.write(current_row, col_idx, row_data[col_name])
        current_row += 1

    # Insert a blank row between groups.
    current_row += 1

workbook.close()
print(f"Ranking 14 successfully saved to {output_path}")


Ranking 14 successfully saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 14.xlsx


In [ ]:
# Install required package if needed.
!pip install XlsxWriter openpyxl

import pandas as pd
import xlsxwriter

# ---------------------------
# Helper Function: Parse Ranking File with Group Headers
# ---------------------------
def parse_ranking_file(filepath):
    """
    Parses a ranking file that uses group headers.
    Each group starts with a cell in Column A that reads "Top 30 Stocks of YYYYMM",
    followed by a row with column headers and then the data rows.
    Returns a dictionary mapping each Timestamp (YYYYMM) to a DataFrame containing that group's data.
    """
    df = pd.read_excel(filepath, header=None, engine='openpyxl')
    groups = {}
    current_timestamp = None
    current_columns = None
    data_rows = []

    for _, row in df.iterrows():
        cell_val = str(row[0]) if pd.notnull(row[0]) else ""
        if cell_val.startswith("Top 30 Stocks of "):
            if current_timestamp is not None and current_columns is not None and data_rows:
                groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
            current_timestamp = cell_val.split("Top 30 Stocks of ")[1].strip()
            current_columns = None
            data_rows = []
        elif current_timestamp is not None and current_columns is None:
            current_columns = row.dropna().tolist()
        elif current_timestamp is not None and current_columns is not None:
            if not row.isnull().all():
                # Only take as many cells as there are headers.
                row_data = row.tolist()[:len(current_columns)]
                data_rows.append(row_data)
    if current_timestamp is not None and current_columns is not None and data_rows:
        groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
    return groups

# ---------------------------
# Step 1. Read the Ranking 10 File and Parse Its Groups
# ---------------------------
ranking10_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 10.xlsx'
ranking10_groups = parse_ranking_file(ranking10_path)

# ---------------------------
# Step 2. Read the Final 25 File and Build a Lookup Dictionary
# ---------------------------
final25_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Final 25.xlsx'
df25 = pd.read_excel(final25_path)

# Create a Timestamp column if not already present (assuming Year and Month exist)
if 'Timestamp' not in df25.columns and 'Year' in df25.columns and 'Month' in df25.columns:
    df25['Timestamp'] = df25['Year'].astype(str) + df25['Month'].apply(lambda x: f"{int(x):02d}")

# Sort by Return_Next 3M descending and drop duplicates for each (Timestamp, PERMNO)
df25 = df25.sort_values('Return_Next 3M', ascending=False)
df25_unique = df25.drop_duplicates(subset=['Timestamp', 'PERMNO'], keep='first')

# Build a lookup dictionary: (Timestamp, PERMNO) -> Return_Next 3M
final25_lookup = {(row['Timestamp'], row['PERMNO']): row['Return_Next 3M']
                  for _, row in df25_unique.iterrows()}

# ---------------------------
# Step 3. Update Ranking 10 Groups Using Final 25 Data
# ---------------------------
# Ranking 10 is expected to have the following columns:
# ["Timestamp", "RollingOLS Single-scaled 2 Rank", "Predicted_Return_Next_3M", "PERMNO", "Ticker", "Company", "Sector"]
# We'll update the predicted values to the corresponding Return_Next 3M from Final 25,
# then rename the column to "Return_Next 3M", and recalc the total row.
for ts, group_df in ranking10_groups.items():
    # Process each data row that is not the total row.
    for idx, row in group_df.iterrows():
        if row["RollingOLS Single-scaled 2 Rank"] != "Total Return":
            key = (row["Timestamp"], row["PERMNO"])
            if key in final25_lookup:
                group_df.at[idx, "Predicted_Return_Next_3M"] = final25_lookup[key]
    # Rename the column in this group.
    group_df.rename(columns={"Predicted_Return_Next_3M": "Return_Next 3M"}, inplace=True)

    # Recalculate total: sum the updated "Return_Next 3M" values for rows where rank is not "Total Return".
    numeric_rows = group_df[group_df["RollingOLS Single-scaled 2 Rank"] != "Total Return"]
    # Convert to float if needed.
    total_val = numeric_rows["Return_Next 3M"].astype(float).sum()
    # Find the total row (where rank equals "Total Return") and update its value.
    total_row = group_df[group_df["RollingOLS Single-scaled 2 Rank"] == "Total Return"]
    if not total_row.empty:
        total_idx = total_row.index[0]
        group_df.at[total_idx, "Return_Next 3M"] = total_val
    ranking10_groups[ts] = group_df

# ---------------------------
# Step 4. Write the Updated Groups to "Ranking 15.xlsx"
# ---------------------------
output_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 15.xlsx'
workbook = xlsxwriter.Workbook(output_path, {'nan_inf_to_errors': True})
worksheet = workbook.add_worksheet("Ranking")

# Define the output column headers (after renaming).
columns = ["Timestamp", "RollingOLS Single-scaled 2 Rank", "Return_Next 3M",
           "PERMNO", "Ticker", "Company", "Sector"]

current_row = 0
# Process groups in sorted order by Timestamp.
for ts, group_df in sorted(ranking10_groups.items(), key=lambda x: x[0]):
    # Write a header row for this group.
    header_text = f"Top 30 Stocks of {ts}"
    worksheet.write(current_row, 0, header_text)
    current_row += 1

    # Write the column headers.
    for col_idx, col_name in enumerate(columns):
        worksheet.write(current_row, col_idx, col_name)
    current_row += 1

    # Write the data rows.
    for _, row_data in group_df.iterrows():
        for col_idx, col_name in enumerate(columns):
            worksheet.write(current_row, col_idx, row_data[col_name])
        current_row += 1

    # Insert a blank row between groups.
    current_row += 1

workbook.close()
print(f"Ranking 15 successfully saved to {output_path}")


Ranking 15 successfully saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 15.xlsx


In [ ]:
# Install required packages if not already installed.
!pip install XlsxWriter openpyxl

import pandas as pd
import xlsxwriter

# ---------------------------
# Helper Function: Parse a Ranking File with Group Headers
# ---------------------------
def parse_ranking_file(filepath):
    """
    Parses a ranking file with group headers.
    Each group starts with a cell in Column A that reads "Top 30 Stocks of YYYYMM",
    followed by a row with column headers and then the data rows.
    Returns a dictionary mapping each Timestamp (YYYYMM) to a DataFrame containing that group's data.
    """
    df = pd.read_excel(filepath, header=None, engine='openpyxl')
    groups = {}
    current_timestamp = None
    current_columns = None
    data_rows = []

    for _, row in df.iterrows():
        cell_val = str(row[0]) if pd.notnull(row[0]) else ""
        if cell_val.startswith("Top 30 Stocks of "):
            if current_timestamp is not None and current_columns is not None and data_rows:
                groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
            current_timestamp = cell_val.split("Top 30 Stocks of ")[1].strip()
            current_columns = None
            data_rows = []
        elif current_timestamp is not None and current_columns is None:
            current_columns = row.dropna().tolist()
        elif current_timestamp is not None and current_columns is not None:
            if not row.isnull().all():
                row_data = row.tolist()[:len(current_columns)]
                data_rows.append(row_data)
    if current_timestamp is not None and current_columns is not None and data_rows:
        groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
    return groups

# ---------------------------
# Step 1. Read Ranking 11 File and Parse Its Groups
# ---------------------------
ranking11_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 11.xlsx'
ranking11_groups = parse_ranking_file(ranking11_path)

# ---------------------------
# Step 2. Read Final 25 File and Build a Lookup Dictionary
# ---------------------------
final25_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Final 25.xlsx'
df25 = pd.read_excel(final25_path)

# Create a Timestamp column if not already present (assuming Year and Month exist)
if 'Timestamp' not in df25.columns and 'Year' in df25.columns and 'Month' in df25.columns:
    df25['Timestamp'] = df25['Year'].astype(str) + df25['Month'].apply(lambda x: f"{int(x):02d}")

# Sort by Return_Next 3M descending and drop duplicates by (Timestamp, PERMNO)
df25 = df25.sort_values('Return_Next 3M', ascending=False)
df25_unique = df25.drop_duplicates(subset=['Timestamp', 'PERMNO'], keep='first')

# Build a lookup dictionary: key = (Timestamp, PERMNO), value = Return_Next 3M
final25_lookup = {(row['Timestamp'], row['PERMNO']): row['Return_Next 3M']
                  for _, row in df25_unique.iterrows()}

# ---------------------------
# Step 3. Update Ranking 11 Groups Using Final 25 Data
# ---------------------------
# Ranking 11 is expected to have the following columns:
# ["Timestamp", "RollingOLS Double-scaled Rank", "Predicted_Return_Next_3M", "PERMNO", "Ticker", "Company", "Sector"]
# For every data row (i.e. rows where "RollingOLS Double-scaled Rank" is not "Total Return"),
# we replace the value in "Predicted_Return_Next_3M" with the corresponding Return_Next 3M from Final 25.
for ts, group_df in ranking11_groups.items():
    for idx, row in group_df.iterrows():
        if row["RollingOLS Double-scaled Rank"] != "Total Return":
            key = (row["Timestamp"], row["PERMNO"])
            if key in final25_lookup:
                group_df.at[idx, "Predicted_Return_Next_3M"] = final25_lookup[key]
    # Rename the column "Predicted_Return_Next_3M" to "Return_Next 3M"
    group_df.rename(columns={"Predicted_Return_Next_3M": "Return_Next 3M"}, inplace=True)

    # Recalculate the total: sum the updated "Return_Next 3M" values for data rows (exclude the total row)
    numeric_rows = group_df[group_df["RollingOLS Double-scaled Rank"] != "Total Return"]
    total_val = numeric_rows["Return_Next 3M"].astype(float).sum()
    # Update the total row if it exists
    total_row = group_df[group_df["RollingOLS Double-scaled Rank"] == "Total Return"]
    if not total_row.empty:
        total_idx = total_row.index[0]
        group_df.at[total_idx, "Return_Next 3M"] = total_val
    ranking11_groups[ts] = group_df

# ---------------------------
# Step 4. Write the Updated Groups to "Ranking 16.xlsx"
# ---------------------------
output_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 16.xlsx'
workbook = xlsxwriter.Workbook(output_path, {'nan_inf_to_errors': True})
worksheet = workbook.add_worksheet("Ranking")

# Define the column headers for the output file.
# After renaming, the columns should be:
# ["Timestamp", "RollingOLS Double-scaled Rank", "Return_Next 3M", "PERMNO", "Ticker", "Company", "Sector"]
columns = ["Timestamp", "RollingOLS Double-scaled Rank", "Return_Next 3M", "PERMNO", "Ticker", "Company", "Sector"]

current_row = 0
# Process groups in sorted order by Timestamp.
for ts, group_df in sorted(ranking11_groups.items(), key=lambda x: x[0]):
    # Write a header row for the group.
    header_text = f"Top 30 Stocks of {ts}"
    worksheet.write(current_row, 0, header_text)
    current_row += 1

    # Write the column headers.
    for col_idx, col_name in enumerate(columns):
        worksheet.write(current_row, col_idx, col_name)
    current_row += 1

    # Write each data row for this group.
    for _, row_data in group_df.iterrows():
        for col_idx, col_name in enumerate(columns):
            worksheet.write(current_row, col_idx, row_data[col_name])
        current_row += 1

    # Insert a blank row between groups.
    current_row += 1

workbook.close()
print(f"Ranking 16 successfully saved to {output_path}")


Ranking 16 successfully saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 16.xlsx


In [ ]:
# Install required packages if not already installed.
!pip install XlsxWriter openpyxl

import pandas as pd
import xlsxwriter

# ---------------------------
# Helper Function: Parse a Ranking File with Group Headers
# ---------------------------
def parse_ranking_file(filepath):
    """
    Parses a ranking file with group headers.
    Each group starts with a cell in Column A that reads "Top 30 Stocks of YYYYMM",
    followed by a row with column headers and then the data rows.
    Returns a dictionary mapping each Timestamp (YYYYMM) to a DataFrame containing that group's data.
    """
    df = pd.read_excel(filepath, header=None, engine='openpyxl')
    groups = {}
    current_timestamp = None
    current_columns = None
    data_rows = []

    for _, row in df.iterrows():
        cell_val = str(row[0]) if pd.notnull(row[0]) else ""
        if cell_val.startswith("Top 30 Stocks of "):
            if current_timestamp is not None and current_columns is not None and data_rows:
                groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
            current_timestamp = cell_val.split("Top 30 Stocks of ")[1].strip()
            current_columns = None
            data_rows = []
        elif current_timestamp is not None and current_columns is None:
            current_columns = row.dropna().tolist()
        elif current_timestamp is not None and current_columns is not None:
            if not row.isnull().all():
                row_data = row.tolist()[:len(current_columns)]
                data_rows.append(row_data)
    if current_timestamp is not None and current_columns is not None and data_rows:
        groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
    return groups

# ---------------------------
# Step 1. Read the Ranking 12 File and Parse Its Groups
# ---------------------------
ranking12_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 12.xlsx'
ranking12_groups = parse_ranking_file(ranking12_path)

# ---------------------------
# Step 2. Read the Final 25 File and Build a Lookup Dictionary
# ---------------------------
final25_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Final 25.xlsx'
df25 = pd.read_excel(final25_path)

# Create a Timestamp column if not already present (assuming Year and Month exist)
if 'Timestamp' not in df25.columns and 'Year' in df25.columns and 'Month' in df25.columns:
    df25['Timestamp'] = df25['Year'].astype(str) + df25['Month'].apply(lambda x: f"{int(x):02d}")

# Sort by "Return_Next 3M" descending and drop duplicates (keeping the highest) for each (Timestamp, PERMNO)
df25 = df25.sort_values('Return_Next 3M', ascending=False)
df25_unique = df25.drop_duplicates(subset=['Timestamp', 'PERMNO'], keep='first')

# Build a lookup dictionary: key = (Timestamp, PERMNO), value = Return_Next 3M
final25_lookup = {
    (row['Timestamp'], row['PERMNO']): row['Return_Next 3M']
    for _, row in df25_unique.iterrows()
}

# ---------------------------
# Step 3. Update Ranking 12 Groups Using Final 25 Data
# ---------------------------
# Ranking 12 is expected to have columns:
# ["Timestamp", "Logistic Single-scaled Rank", "predicted_prob", "PERMNO", "Ticker", "Company", "Sector"]
for ts, group_df in ranking12_groups.items():
    # For each data row (skip the total row marked "Total Return")
    for idx, row in group_df.iterrows():
        if row["Logistic Single-scaled Rank"] != "Total Return":
            key = (row["Timestamp"], row["PERMNO"])
            if key in final25_lookup:
                group_df.at[idx, "predicted_prob"] = final25_lookup[key]
    # Rename the column from "predicted_prob" to "Return_Next 3M"
    group_df.rename(columns={"predicted_prob": "Return_Next 3M"}, inplace=True)

    # Recalculate the total: sum of the updated "Return_Next 3M" values (for rows where rank is not "Total Return")
    numeric_rows = group_df[group_df["Logistic Single-scaled Rank"] != "Total Return"]
    total_val = numeric_rows["Return_Next 3M"].astype(float).sum()
    # Update the total row (if exists)
    total_row = group_df[group_df["Logistic Single-scaled Rank"] == "Total Return"]
    if not total_row.empty:
        total_idx = total_row.index[0]
        group_df.at[total_idx, "Return_Next 3M"] = total_val
    ranking12_groups[ts] = group_df

# ---------------------------
# Step 4. Write the Updated Groups to "Ranking 17.xlsx"
# ---------------------------
output_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 17.xlsx'
workbook = xlsxwriter.Workbook(output_path, {'nan_inf_to_errors': True})
worksheet = workbook.add_worksheet("Ranking")

# Define the output column headers.
# After renaming, the columns are:
# ["Timestamp", "Logistic Single-scaled Rank", "Return_Next 3M", "PERMNO", "Ticker", "Company", "Sector"]
columns = ["Timestamp", "Logistic Single-scaled Rank", "Return_Next 3M", "PERMNO", "Ticker", "Company", "Sector"]

current_row = 0
# Process groups in sorted order by Timestamp.
for ts, group_df in sorted(ranking12_groups.items(), key=lambda x: x[0]):
    # Write a header row for the group.
    header_text = f"Top 30 Stocks of {ts}"
    worksheet.write(current_row, 0, header_text)
    current_row += 1

    # Write the column headers.
    for col_idx, col_name in enumerate(columns):
        worksheet.write(current_row, col_idx, col_name)
    current_row += 1

    # Write each data row for this group.
    for _, row_data in group_df.iterrows():
        for col_idx, col_name in enumerate(columns):
            worksheet.write(current_row, col_idx, row_data[col_name])
        current_row += 1

    # Insert a blank row between groups.
    current_row += 1

workbook.close()
print(f"Ranking 17 successfully saved to {output_path}")


Ranking 17 successfully saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 17.xlsx


In [ ]:
# Install required packages if not already installed.
!pip install XlsxWriter openpyxl

import pandas as pd
import xlsxwriter

# ---------------------------
# Helper Function: Parse a Ranking File with Group Headers
# ---------------------------
def parse_ranking_file(filepath):
    """
    Parses a ranking file that uses group headers.
    Each group starts with a cell in Column A that reads "Top 30 Stocks of YYYYMM",
    followed by a row with column headers and then the data rows.
    Returns a dictionary mapping each Timestamp (YYYYMM) to a DataFrame containing that group's data.
    """
    df = pd.read_excel(filepath, header=None, engine='openpyxl')
    groups = {}
    current_timestamp = None
    current_columns = None
    data_rows = []

    for _, row in df.iterrows():
        cell_val = str(row[0]) if pd.notnull(row[0]) else ""
        if cell_val.startswith("Top 30 Stocks of "):
            if current_timestamp is not None and current_columns is not None and data_rows:
                groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
            current_timestamp = cell_val.split("Top 30 Stocks of ")[1].strip()
            current_columns = None
            data_rows = []
        elif current_timestamp is not None and current_columns is None:
            current_columns = row.dropna().tolist()
        elif current_timestamp is not None and current_columns is not None:
            if not row.isnull().all():
                row_data = row.tolist()[:len(current_columns)]
                data_rows.append(row_data)
    if current_timestamp is not None and current_columns is not None and data_rows:
        groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
    return groups

# ---------------------------
# Step 1. Read the Ranking 13 File and Parse Its Groups
# ---------------------------
ranking13_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 13.xlsx'
ranking13_groups = parse_ranking_file(ranking13_path)

# ---------------------------
# Step 2. Read the Final 25 File and Build a Lookup Dictionary
# ---------------------------
final25_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Final 25.xlsx'
df25 = pd.read_excel(final25_path)

# Create a Timestamp column if not already present (assuming Year and Month exist)
if 'Timestamp' not in df25.columns and 'Year' in df25.columns and 'Month' in df25.columns:
    df25['Timestamp'] = df25['Year'].astype(str) + df25['Month'].apply(lambda x: f"{int(x):02d}")

# Sort by Return_Next 3M descending and drop duplicates by (Timestamp, PERMNO)
df25 = df25.sort_values('Return_Next 3M', ascending=False)
df25_unique = df25.drop_duplicates(subset=['Timestamp', 'PERMNO'], keep='first')

# Build a lookup dictionary: key = (Timestamp, PERMNO), value = Return_Next 3M
final25_lookup = {(row['Timestamp'], row['PERMNO']): row['Return_Next 3M']
                  for _, row in df25_unique.iterrows()}

# ---------------------------
# Step 3. Update Ranking 13 Groups Using Final 25 Data
# ---------------------------
# Ranking 13 is expected to have the following columns:
# ["Timestamp", "Logistic Double-scaled Rank", "predicted_prob", "PERMNO", "Ticker", "Company", "Sector"]
for ts, group_df in ranking13_groups.items():
    # For each data row (skip the total row marked by "Total Return")
    for idx, row in group_df.iterrows():
        if row["Logistic Double-scaled Rank"] != "Total Return":
            key = (row["Timestamp"], row["PERMNO"])
            if key in final25_lookup:
                group_df.at[idx, "predicted_prob"] = final25_lookup[key]
    # Rename the column from "predicted_prob" to "Return_Next 3M"
    group_df.rename(columns={"predicted_prob": "Return_Next 3M"}, inplace=True)

    # Recalculate the total: sum of the updated "Return_Next 3M" values for data rows (excluding the total row)
    numeric_rows = group_df[group_df["Logistic Double-scaled Rank"] != "Total Return"]
    total_val = numeric_rows["Return_Next 3M"].astype(float).sum()
    # Update the total row (if it exists)
    total_row = group_df[group_df["Logistic Double-scaled Rank"] == "Total Return"]
    if not total_row.empty:
        total_idx = total_row.index[0]
        group_df.at[total_idx, "Return_Next 3M"] = total_val
    ranking13_groups[ts] = group_df

# ---------------------------
# Step 4. Write the Updated Groups to "Ranking 18.xlsx"
# ---------------------------
output_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 18.xlsx'
workbook = xlsxwriter.Workbook(output_path, {'nan_inf_to_errors': True})
worksheet = workbook.add_worksheet("Ranking")

# Define the output column headers.
# After renaming, the columns are:
# ["Timestamp", "Logistic Double-scaled Rank", "Return_Next 3M", "PERMNO", "Ticker", "Company", "Sector"]
columns = ["Timestamp", "Logistic Double-scaled Rank", "Return_Next 3M", "PERMNO", "Ticker", "Company", "Sector"]

current_row = 0
# Process groups in sorted order by Timestamp.
for ts, group_df in sorted(ranking13_groups.items(), key=lambda x: x[0]):
    # Write a header row for the group.
    header_text = f"Top 30 Stocks of {ts}"
    worksheet.write(current_row, 0, header_text)
    current_row += 1

    # Write the column headers.
    for col_idx, col_name in enumerate(columns):
        worksheet.write(current_row, col_idx, col_name)
    current_row += 1

    # Write each data row for this group.
    for _, row_data in group_df.iterrows():
        for col_idx, col_name in enumerate(columns):
            worksheet.write(current_row, col_idx, row_data[col_name])
        current_row += 1

    # Insert a blank row between groups.
    current_row += 1

workbook.close()
print(f"Ranking 18 successfully saved to {output_path}")


Ranking 18 successfully saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 18.xlsx


In [ ]:
# Install XlsxWriter if not already installed.
!pip install XlsxWriter

import pandas as pd
import xlsxwriter

# ---------------------------
# Step 1. Read the Final 50 File
# ---------------------------
final50_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Final 50.xlsx'
df50 = pd.read_excel(final50_path)

# Create a Timestamp column if needed (assuming Year and Month exist)
if 'Timestamp' not in df50.columns and 'Year' in df50.columns and 'Month' in df50.columns:
    df50['Timestamp'] = df50['Year'].astype(str) + df50['Month'].apply(lambda x: f"{int(x):02d}")

# ---------------------------
# Step 2. Remove Duplicate Entries
# ---------------------------
# Sort by predicted_prob descending and drop duplicates based on (Timestamp, PERMNO)
df50 = df50.sort_values('predicted_prob', ascending=False)
df50_unique = df50.drop_duplicates(subset=['Timestamp', 'PERMNO'], keep='first')

# ---------------------------
# Step 3. Merge in Company Info from Company 1 File
# ---------------------------
company1_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Company 1.xlsx'
df_company = pd.read_excel(company1_path)
df50_merged = pd.merge(df50_unique,
                       df_company[['PERMNO', 'Ticker', 'Company', 'Sector']],
                       on='PERMNO',
                       how='left')

# ---------------------------
# Step 4. Group by Timestamp and Rank Top 30 Winners
# ---------------------------
grouped_data = []
for ts, group in df50_merged.groupby('Timestamp'):
    # Sort the group by predicted_prob (highest first) and take the top 30 rows.
    group_sorted = group.sort_values(by='predicted_prob', ascending=False).head(30).copy()
    group_sorted = group_sorted.reset_index(drop=True)
    group_sorted['Random Forest Rank'] = range(1, len(group_sorted) + 1)

    # Keep only the required columns in the specified order.
    group_sorted = group_sorted[['Timestamp', 'Random Forest Rank', 'predicted_prob', 'PERMNO', 'Ticker', 'Company', 'Sector']]
    grouped_data.append((ts, group_sorted))

# ---------------------------
# Step 5. Write the Output to "Ranking 19.xlsx" with Group Headers and a Total Row
# ---------------------------
output_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 19.xlsx'
workbook = xlsxwriter.Workbook(output_path, {'nan_inf_to_errors': True})
worksheet = workbook.add_worksheet("Ranking")

# Define the output column headers.
columns = ["Timestamp", "Random Forest Rank", "predicted_prob", "PERMNO", "Ticker", "Company", "Sector"]

current_row = 0
# Process each group sorted by Timestamp.
for ts, group_df in sorted(grouped_data, key=lambda x: x[0]):
    # Write a header row for this group.
    header_text = f"Top 30 Stocks of {ts}"
    worksheet.write(current_row, 0, header_text)
    current_row += 1

    # Write the column headers.
    for col_idx, col_name in enumerate(columns):
        worksheet.write(current_row, col_idx, col_name)
    current_row += 1

    # Write each data row for this Timestamp group.
    for _, row_data in group_df.iterrows():
        for col_idx, col_name in enumerate(columns):
            worksheet.write(current_row, col_idx, row_data[col_name])
        current_row += 1

    # Add a total row: Sum the predicted_prob values for the group.
    total_val = group_df['predicted_prob'].astype(float).sum()
    worksheet.write(current_row, 1, "Total Return")
    worksheet.write(current_row, 2, total_val)
    current_row += 1

    # Insert a blank row between groups.
    current_row += 1

workbook.close()
print(f"Ranking 19 successfully saved to {output_path}")


Ranking 19 successfully saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 19.xlsx


In [ ]:
# Install required packages if not already installed.
!pip install XlsxWriter openpyxl

import pandas as pd
import xlsxwriter

# ---------------------------
# Helper Function: Parse a Ranking File with Group Headers
# ---------------------------
def parse_ranking_file(filepath):
    """
    Parses a ranking file with group headers.
    Each group starts with a cell in Column A that reads "Top 30 Stocks of YYYYMM",
    followed by a row with column headers and then the data rows.
    Returns a dictionary mapping each Timestamp (YYYYMM) to a DataFrame containing that group's data.
    """
    df = pd.read_excel(filepath, header=None, engine='openpyxl')
    groups = {}
    current_timestamp = None
    current_columns = None
    data_rows = []

    for _, row in df.iterrows():
        cell_val = str(row[0]) if pd.notnull(row[0]) else ""
        if cell_val.startswith("Top 30 Stocks of "):
            if current_timestamp is not None and current_columns is not None and data_rows:
                groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
            current_timestamp = cell_val.split("Top 30 Stocks of ")[1].strip()
            current_columns = None
            data_rows = []
        elif current_timestamp is not None and current_columns is None:
            current_columns = row.dropna().tolist()
        elif current_timestamp is not None and current_columns is not None:
            if not row.isnull().all():
                row_data = row.tolist()[:len(current_columns)]
                data_rows.append(row_data)
    if current_timestamp is not None and current_columns is not None and data_rows:
        groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
    return groups

# ---------------------------
# Step 1. Read the Ranking 19 File and Parse Its Groups
# ---------------------------
ranking19_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 19.xlsx'
ranking19_groups = parse_ranking_file(ranking19_path)

# ---------------------------
# Step 2. Read the Final 25 File and Build a Lookup Dictionary
# ---------------------------
final25_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Final 25.xlsx'
df25 = pd.read_excel(final25_path)

# Create a Timestamp column if not present (assuming Year and Month exist)
if 'Timestamp' not in df25.columns and 'Year' in df25.columns and 'Month' in df25.columns:
    df25['Timestamp'] = df25['Year'].astype(str) + df25['Month'].apply(lambda x: f"{int(x):02d}")

# Sort by "Return_Next 3M" descending and drop duplicates for each (Timestamp, PERMNO)
df25 = df25.sort_values('Return_Next 3M', ascending=False)
df25_unique = df25.drop_duplicates(subset=['Timestamp', 'PERMNO'], keep='first')

# Build a lookup dictionary: key = (Timestamp, PERMNO), value = Return_Next 3M
final25_lookup = {(row['Timestamp'], row['PERMNO']): row['Return_Next 3M']
                  for _, row in df25_unique.iterrows()}

# ---------------------------
# Step 3. Update Ranking 19 Groups Using Final 25 Data
# ---------------------------
# Ranking 19 is expected to have columns:
# ["Timestamp", "Random Forest Rank", "predicted_prob", "PERMNO", "Ticker", "Company", "Sector"]
for ts, group_df in ranking19_groups.items():
    # For every data row that is not the total row (assumed marked by "Total Return" in the rank column)
    for idx, row in group_df.iterrows():
        if row["Random Forest Rank"] != "Total Return":
            key = (row["Timestamp"], row["PERMNO"])
            if key in final25_lookup:
                group_df.at[idx, "predicted_prob"] = final25_lookup[key]
    # Rename the column "predicted_prob" to "Return_Next 3M"
    group_df.rename(columns={"predicted_prob": "Return_Next 3M"}, inplace=True)

    # Recalculate total: sum of updated "Return_Next 3M" values (for rows where rank is not "Total Return")
    numeric_rows = group_df[group_df["Random Forest Rank"] != "Total Return"]
    total_val = numeric_rows["Return_Next 3M"].astype(float).sum()
    # Update the total row (if it exists)
    total_row = group_df[group_df["Random Forest Rank"] == "Total Return"]
    if not total_row.empty:
        total_idx = total_row.index[0]
        group_df.at[total_idx, "Return_Next 3M"] = total_val
    ranking19_groups[ts] = group_df

# ---------------------------
# Step 4. Write the Updated Groups to "Ranking 20.xlsx"
# ---------------------------
output_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 20.xlsx'
workbook = xlsxwriter.Workbook(output_path, {'nan_inf_to_errors': True})
worksheet = workbook.add_worksheet("Ranking")

# Define the output column headers.
# The final columns will be:
# ["Timestamp", "Random Forest Rank", "Return_Next 3M", "PERMNO", "Ticker", "Company", "Sector"]
columns = ["Timestamp", "Random Forest Rank", "Return_Next 3M", "PERMNO", "Ticker", "Company", "Sector"]

current_row = 0
# Write groups in sorted order by Timestamp.
for ts, group_df in sorted(ranking19_groups.items(), key=lambda x: x[0]):
    # Write a header row for the group.
    header_text = f"Top 30 Stocks of {ts}"
    worksheet.write(current_row, 0, header_text)
    current_row += 1

    # Write the column headers.
    for col_idx, col_name in enumerate(columns):
        worksheet.write(current_row, col_idx, col_name)
    current_row += 1

    # Write each row of data for this group.
    for _, row_data in group_df.iterrows():
        for col_idx, col_name in enumerate(columns):
            worksheet.write(current_row, col_idx, row_data[col_name])
        current_row += 1

    # Insert a blank row between groups.
    current_row += 1

workbook.close()
print(f"Ranking 20 successfully saved to {output_path}")


Ranking 20 successfully saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 20.xlsx


In [ ]:
# Install required packages if not already installed.
!pip install XlsxWriter openpyxl

import pandas as pd
import xlsxwriter

def parse_ranking_file(filepath):
    """
    Parses a ranking file that uses group headers.
    Each group starts with a cell in column A that reads "Top 30 Stocks of YYYYMM",
    followed by a row of column headers and then the data rows.
    Returns a dictionary mapping each Timestamp (YYYYMM) to a DataFrame containing that group's data.
    """
    df = pd.read_excel(filepath, header=None, engine='openpyxl')
    groups = {}
    current_timestamp = None
    current_columns = None
    data_rows = []

    for _, row in df.iterrows():
        cell_val = str(row[0]) if pd.notnull(row[0]) else ""
        if cell_val.startswith("Top 30 Stocks of "):
            # When a new group header is found, save the previous group (if any)
            if current_timestamp is not None and current_columns is not None and data_rows:
                groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
            # Extract the Timestamp (YYYYMM) from the header
            current_timestamp = cell_val.split("Top 30 Stocks of ")[1].strip()
            current_columns = None
            data_rows = []
        elif current_timestamp is not None and current_columns is None:
            # The row immediately after the header is assumed to be the column headers
            current_columns = row.dropna().tolist()
        elif current_timestamp is not None and current_columns is not None:
            # Subsequent non-empty rows are treated as data rows.
            if not row.isnull().all():
                row_data = row.tolist()[:len(current_columns)]
                data_rows.append(row_data)
    # Save the last group
    if current_timestamp is not None and current_columns is not None and data_rows:
        groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
    return groups

# Define the list of ranking files in the desired sequence.
files_order = [
    ("Ranking 6", "/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 6.xlsx"),
    ("Ranking 14", "/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 14.xlsx"),
    ("Ranking 15", "/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 15.xlsx"),
    ("Ranking 16", "/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 16.xlsx"),
    ("Ranking 17", "/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 17.xlsx"),
    ("Ranking 18", "/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 18.xlsx"),
    ("Ranking 20", "/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 20.xlsx")
]

# Parse each file and store the groups in a dictionary keyed by the file's name.
parsed = {}
for name, path in files_order:
    parsed[name] = parse_ranking_file(path)

# We'll use Ranking 6 as the base. For each Timestamp in Ranking 6,
# if matching groups exist in the other files, horizontally concatenate them.
combined_groups = {}
base_name = "Ranking 6"
for ts in sorted(parsed[base_name].keys()):
    # Start with the Ranking 6 group for this Timestamp.
    combined_df = parsed[base_name][ts].reset_index(drop=True)
    # Iterate over the remaining files in the specified order.
    for name, _ in files_order[1:]:
        if ts in parsed[name]:
            df_temp = parsed[name][ts].reset_index(drop=True)
            # Concatenate horizontally (i.e., add the columns from df_temp to the right).
            combined_df = pd.concat([combined_df, df_temp], axis=1)
    combined_groups[ts] = combined_df

# Write the combined groups to "Ranking 21.xlsx"
output_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 21.xlsx'
workbook = xlsxwriter.Workbook(output_path, {'nan_inf_to_errors': True})
worksheet = workbook.add_worksheet("Combined Ranking")

current_row = 0

# Write each Timestamp group in sorted order.
for ts in sorted(combined_groups.keys()):
    # Write a header row for this group.
    header_text = f"Top 30 Stocks of {ts}"
    worksheet.write(current_row, 0, header_text)
    current_row += 1

    # Write the column headers (from the combined DataFrame).
    combined_df = combined_groups[ts]
    for col_idx, col_name in enumerate(combined_df.columns):
        worksheet.write(current_row, col_idx, col_name)
    current_row += 1

    # Write all data rows.
    for _, row_data in combined_df.iterrows():
        for col_idx, cell in enumerate(row_data):
            worksheet.write(current_row, col_idx, cell)
        current_row += 1

    # Insert a blank row between groups for readability.
    current_row += 1

workbook.close()
print(f"Combined file successfully saved to {output_path} as Ranking 21.xlsx")


Combined file successfully saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 21.xlsx as Ranking 21.xlsx


In [ ]:
# Install required packages if not already installed.
!pip install XlsxWriter openpyxl plotly

import pandas as pd
import xlsxwriter
import plotly.graph_objects as go

def parse_ranking_file(filepath):
    """
    Parses a ranking file that uses group headers.
    Each group starts with a cell in column A reading "Top 30 Stocks of YYYYMM",
    followed by a row of column headers and then the data rows.
    Returns a dictionary mapping each Timestamp (YYYYMM) to a DataFrame of that group's data.
    """
    df = pd.read_excel(filepath, header=None, engine='openpyxl')
    groups = {}
    current_timestamp = None
    current_columns = None
    data_rows = []

    for _, row in df.iterrows():
        cell_val = str(row[0]) if pd.notnull(row[0]) else ""
        if cell_val.startswith("Top 30 Stocks of "):
            # Save previous group if any
            if current_timestamp is not None and current_columns is not None and data_rows:
                groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
            current_timestamp = cell_val.split("Top 30 Stocks of ")[1].strip()
            current_columns = None
            data_rows = []
        elif current_timestamp is not None and current_columns is None:
            # Next row is assumed to be column headers
            current_columns = row.dropna().tolist()
        elif current_timestamp is not None and current_columns is not None:
            if not row.isnull().all():
                row_data = row.tolist()[:len(current_columns)]
                data_rows.append(row_data)
    if current_timestamp is not None and current_columns is not None and data_rows:
        groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
    return groups

# ---------------------------
# Step 1. Read Ranking 21 and parse its groups.
# ---------------------------
ranking21_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 21.xlsx'
ranking21_groups = parse_ranking_file(ranking21_path)

# ---------------------------
# Step 2. Extract Total Return values for each model in each Timestamp group.
# ---------------------------
# We assume each ranking block is 7 columns wide:
# [Timestamp, <Rank Title>, <Return Column>, PERMNO, Ticker, Company, Sector]
# The total row is assumed to be the last row in the group.
summary_records = []

for ts, df_group in ranking21_groups.items():
    if df_group.shape[0] < 1:
        continue
    total_row = df_group.iloc[-1]  # Last row assumed to be Total Return row
    ncols = df_group.shape[1]
    block_width = 7
    nblocks = ncols // block_width
    for i in range(nblocks):
        block_start = i * block_width
        # The Rank column in this block is at block_start + 1; use its header as model name.
        model_name = df_group.columns[block_start + 1]
        # The Return column for this block is at block_start + 2.
        total_return = total_row[block_start + 2]
        summary_records.append({
            "Timestamp": ts,
            "Model": model_name,
            "Total Return": total_return
        })

df_summary = pd.DataFrame(summary_records)
df_summary = df_summary.sort_values("Timestamp")

# ---------------------------
# Step 3. Create a fixed (non-animated) chart with full screen functionality.
# ---------------------------
models = df_summary["Model"].unique()
timestamps = sorted(df_summary["Timestamp"].unique())

traces = []
for model in models:
    df_model = df_summary[df_summary["Model"] == model]
    ret_dict = dict(zip(df_model["Timestamp"], df_model["Total Return"]))
    y_vals = [ret_dict.get(ts, None) for ts in timestamps]
    traces.append(go.Scatter(
        x=timestamps,
        y=y_vals,
        mode='lines+markers',
        name=model
    ))

fig = go.Figure(
    data=traces,
    layout=go.Layout(
        title="Total Return by Model Over Time",
        xaxis=dict(title="Timestamp"),
        yaxis=dict(title="Total Return")
    )
)

# Configuration to add a full-screen toggle button in the mode bar.
config = {
    'modeBarButtonsToAdd': ['toggleFullScreen'],
    'displayModeBar': True,
    'responsive': True
}

fig.show(config=config)


<ipython-input-21-0a4ea47debaa>:67: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [ ]:
# Install required packages if not already installed.
!pip install XlsxWriter openpyxl plotly

import pandas as pd
import xlsxwriter
import plotly.graph_objects as go

def parse_ranking_file(filepath):
    """
    Parses a ranking file that uses group headers.
    Each group starts with a cell in column A reading "Top 30 Stocks of YYYYMM",
    followed by a row of column headers and then the data rows.
    Returns a dictionary mapping each Timestamp (YYYYMM) to a DataFrame of that group's data.
    """
    df = pd.read_excel(filepath, header=None, engine='openpyxl')
    groups = {}
    current_timestamp = None
    current_columns = None
    data_rows = []

    for _, row in df.iterrows():
        cell_val = str(row[0]) if pd.notnull(row[0]) else ""
        if cell_val.startswith("Top 30 Stocks of "):
            if current_timestamp is not None and current_columns is not None and data_rows:
                groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
            current_timestamp = cell_val.split("Top 30 Stocks of ")[1].strip()
            current_columns = None
            data_rows = []
        elif current_timestamp is not None and current_columns is None:
            current_columns = row.dropna().tolist()
        elif current_timestamp is not None and current_columns is not None:
            if not row.isnull().all():
                row_data = row.tolist()[:len(current_columns)]
                data_rows.append(row_data)
    if current_timestamp is not None and current_columns is not None and data_rows:
        groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
    return groups

# ---------------------------
# Step 1. Read Ranking 21 and parse its groups.
# ---------------------------
ranking21_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 21.xlsx'
ranking21_groups = parse_ranking_file(ranking21_path)

# ---------------------------
# Step 2. Extract Total Return values for each model in each Timestamp group.
# ---------------------------
# We assume each ranking block is 7 columns wide:
# [Timestamp, <Rank Title>, <Return Column>, PERMNO, Ticker, Company, Sector]
# The total row is assumed to be the last row in the group.
summary_records = []

for ts, df_group in ranking21_groups.items():
    if df_group.shape[0] < 1:
        continue
    total_row = df_group.iloc[-1]  # last row assumed to be the Total Return row
    ncols = df_group.shape[1]
    block_width = 7  # each block is 7 columns
    nblocks = ncols // block_width  # number of ranking blocks
    for i in range(nblocks):
        block_start = i * block_width
        # The Rank column in this block is at block_start + 1; we take its header as the model name.
        model_name = df_group.columns[block_start + 1]
        # The Return column for this block is at block_start + 2.
        total_return = total_row[block_start + 2]
        summary_records.append({
            "Timestamp": ts,
            "Model": model_name,
            "Total Return": total_return
        })

df_summary = pd.DataFrame(summary_records)

# ---------------------------
# Step 3. Filter Timestamps from 200601 to 202309.
# ---------------------------
df_summary = df_summary[(df_summary["Timestamp"] >= "200601") & (df_summary["Timestamp"] <= "202309")]

# Sort the summary by Timestamp.
df_summary = df_summary.sort_values("Timestamp")
timestamps = sorted(df_summary["Timestamp"].unique())

# ---------------------------
# Step 4. Build traces for each model.
# ---------------------------
models = df_summary["Model"].unique()
traces = []
for model in models:
    df_model = df_summary[df_summary["Model"] == model]
    ret_dict = dict(zip(df_model["Timestamp"], df_model["Total Return"]))
    y_vals = [ret_dict.get(ts, None) for ts in timestamps]
    traces.append(go.Scatter(
        x=timestamps,
        y=y_vals,
        mode='lines+markers',
        name=model
    ))

# ---------------------------
# Step 5. Create and display the chart with full screen toggle.
# ---------------------------
fig = go.Figure(
    data=traces,
    layout=go.Layout(
        title="Total Return by Model Over Time (200601 to 202309)",
        xaxis=dict(title="Timestamp"),
        yaxis=dict(title="Total Return")
    )
)

# Add full-screen functionality in the mode bar.
config = {
    'modeBarButtonsToAdd': ['toggleFullScreen'],
    'displayModeBar': True,
    'responsive': True
}

fig.show(config=config)


<ipython-input-22-ca99d44ae073>:65: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [ ]:
# Install required packages if not already installed.
!pip install XlsxWriter openpyxl plotly

import pandas as pd
import xlsxwriter
import plotly.graph_objects as go

def parse_ranking_file(filepath):
    """
    Parses a ranking file that uses group headers.
    Each group starts with a cell in column A reading "Top 30 Stocks of YYYYMM",
    followed by a row of column headers and then the data rows.
    Returns a dictionary mapping each Timestamp (YYYYMM) to a DataFrame of that group's data.
    """
    df = pd.read_excel(filepath, header=None, engine='openpyxl')
    groups = {}
    current_timestamp = None
    current_columns = None
    data_rows = []

    for _, row in df.iterrows():
        cell_val = str(row[0]) if pd.notnull(row[0]) else ""
        if cell_val.startswith("Top 30 Stocks of "):
            if current_timestamp is not None and current_columns is not None and data_rows:
                groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
            current_timestamp = cell_val.split("Top 30 Stocks of ")[1].strip()
            current_columns = None
            data_rows = []
        elif current_timestamp is not None and current_columns is None:
            current_columns = row.dropna().tolist()
        elif current_timestamp is not None and current_columns is not None:
            if not row.isnull().all():
                row_data = row.tolist()[:len(current_columns)]
                data_rows.append(row_data)
    if current_timestamp is not None and current_columns is not None and data_rows:
        groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
    return groups

# ---------------------------
# Step 1. Read Ranking 21 and parse its groups.
# ---------------------------
ranking21_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 21.xlsx'
ranking21_groups = parse_ranking_file(ranking21_path)

# ---------------------------
# Step 2. Extract Total Return values for each model in each Timestamp group.
# ---------------------------
# We assume each ranking block is 7 columns wide:
# [Timestamp, <Rank Title>, <Return Column>, PERMNO, Ticker, Company, Sector]
# The total row is assumed to be the last row in the group.
summary_records = []

for ts, df_group in ranking21_groups.items():
    if df_group.shape[0] < 1:
        continue
    total_row = df_group.iloc[-1]  # last row assumed to be the Total Return row
    ncols = df_group.shape[1]
    block_width = 7  # each block is 7 columns
    nblocks = ncols // block_width  # number of ranking blocks
    for i in range(nblocks):
        block_start = i * block_width
        # Extract model name and remove the trailing " Rank" if present.
        model_name = df_group.columns[block_start + 1]
        model_name = model_name.replace(" Rank", "").strip()
        # The Return column for this block is at block_start + 2.
        total_return = total_row[block_start + 2]
        summary_records.append({
            "Timestamp": ts,
            "Model": model_name,
            "Total Return": total_return
        })

df_summary = pd.DataFrame(summary_records)

# ---------------------------
# Step 3. Filter Timestamps from 200601 to 202309.
# ---------------------------
df_summary = df_summary[(df_summary["Timestamp"] >= "200601") & (df_summary["Timestamp"] <= "202309")]
df_summary = df_summary.sort_values("Timestamp")
timestamps = sorted(df_summary["Timestamp"].unique())

# ---------------------------
# Step 4. Build traces for each model.
# ---------------------------
models = df_summary["Model"].unique()
traces = []
for model in models:
    df_model = df_summary[df_summary["Model"] == model]
    ret_dict = dict(zip(df_model["Timestamp"], df_model["Total Return"]))
    y_vals = [ret_dict.get(ts, None) for ts in timestamps]
    traces.append(go.Scatter(
        x=timestamps,
        y=y_vals,
        mode='lines+markers',
        name=model
    ))

# ---------------------------
# Step 5. Create and display the chart with full screen toggle.
# ---------------------------
fig = go.Figure(
    data=traces,
    layout=go.Layout(
        title="Total Return by Model Over Time (200601 to 202309)",
        xaxis=dict(title="Timestamp"),
        yaxis=dict(title="Total Return")
    )
)

# Configuration to add full-screen toggle functionality.
config = {
    'modeBarButtonsToAdd': ['toggleFullScreen'],
    'displayModeBar': True,
    'responsive': True
}

fig.show(config=config)


<ipython-input-24-ed664b76034e>:66: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [ ]:
# Install required packages if not already installed.
!pip install XlsxWriter openpyxl plotly

import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

def parse_ranking_file(filepath):
    """
    Parses a ranking file that uses group headers.
    Each group starts with a cell in column A reading "Top 30 Stocks of YYYYMM",
    followed by a row of column headers and then the data rows.
    Returns a dictionary mapping each Timestamp (YYYYMM) to a DataFrame of that group's data.
    """
    df = pd.read_excel(filepath, header=None, engine='openpyxl')
    groups = {}
    current_timestamp = None
    current_columns = None
    data_rows = []

    for _, row in df.iterrows():
        cell_val = str(row[0]) if pd.notnull(row[0]) else ""
        if cell_val.startswith("Top 30 Stocks of "):
            if current_timestamp is not None and current_columns is not None and data_rows:
                groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
            current_timestamp = cell_val.split("Top 30 Stocks of ")[1].strip()
            current_columns = None
            data_rows = []
        elif current_timestamp is not None and current_columns is None:
            current_columns = row.dropna().tolist()
        elif current_timestamp is not None and current_columns is not None:
            if not row.isnull().all():
                row_data = row.tolist()[:len(current_columns)]
                data_rows.append(row_data)
    if current_timestamp is not None and current_columns is not None and data_rows:
        groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
    return groups

# ---------------------------
# Step 1. Read Ranking 21 and parse its groups.
# ---------------------------
ranking21_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 21.xlsx'
ranking21_groups = parse_ranking_file(ranking21_path)

# ---------------------------
# Step 2. Extract Total Return values for each model in each Timestamp group.
# ---------------------------
# We assume each ranking block is 7 columns wide:
# [Timestamp, <Rank Title>, <Return Column>, PERMNO, Ticker, Company, Sector]
# The total row is assumed to be the last row in the group.
summary_records = []

for ts, df_group in ranking21_groups.items():
    if df_group.shape[0] < 1:
        continue
    total_row = df_group.iloc[-1]  # Last row assumed to be the Total Return row
    ncols = df_group.shape[1]
    block_width = 7  # each block is 7 columns
    nblocks = ncols // block_width  # number of ranking blocks
    for i in range(nblocks):
        block_start = i * block_width
        # Extract model name from the Rank column header and remove trailing " Rank"
        model_name = df_group.columns[block_start + 1]
        model_name = model_name.replace(" Rank", "").strip()
        # Use .iloc to access by position in the total row
        total_return = total_row.iloc[block_start + 2]
        summary_records.append({
            "Timestamp": ts,
            "Model": model_name,
            "Total Return": total_return
        })

df_summary = pd.DataFrame(summary_records)
df_summary = df_summary.sort_values("Timestamp")

# ---------------------------
# Step 3. Filter Timestamps from 200601 to 202309.
# ---------------------------
df_summary = df_summary[(df_summary["Timestamp"] >= "200601") & (df_summary["Timestamp"] <= "202309")]
df_summary = df_summary.sort_values("Timestamp")
timestamps = sorted(df_summary["Timestamp"].unique())

# ---------------------------
# Step 4. Build traces for each model.
# ---------------------------
models = df_summary["Model"].unique()
traces = []
for model in models:
    df_model = df_summary[df_summary["Model"] == model]
    ret_dict = dict(zip(df_model["Timestamp"], df_model["Total Return"]))
    y_vals = [ret_dict.get(ts, None) for ts in timestamps]
    traces.append(go.Scatter(
        x=timestamps,
        y=y_vals,
        mode='lines+markers',
        name=model
    ))

# ---------------------------
# Step 5. Create and save the chart with full screen toggle.
# ---------------------------
fig = go.Figure(
    data=traces,
    layout=go.Layout(
        title="Total Return by Model Over Time (200601 to 202309)",
        xaxis=dict(title="Timestamp"),
        yaxis=dict(title="Total Return")
    )
)

# Configuration: add full-screen toggle to the mode bar.
config = {
    'modeBarButtonsToAdd': ['toggleFullScreen'],
    'displayModeBar': True,
    'responsive': True
}

# Save as a standalone HTML file named "Chart 2.html"
output_html = "/content/drive/My Drive/Colab Notebooks/823 Project/Chart 2.html"
pio.write_html(fig, file=output_html, auto_open=True, config=config)

print(f"Chart saved to {output_html}. Open this file in a web browser for full-screen functionality.")


Chart saved to /content/drive/My Drive/Colab Notebooks/823 Project/Chart 2.html. Open this file in a web browser for full-screen functionality.


In [ ]:
# Install required packages if not already installed.
!pip install XlsxWriter openpyxl

import pandas as pd
import xlsxwriter

def parse_ranking_file(filepath):
    """
    Parses a ranking file with group headers.
    Each group starts with a cell in column A reading "Top 30 Stocks of YYYYMM",
    followed by a row of column headers and then the data rows.
    Returns a dictionary mapping each Timestamp (YYYYMM) to a DataFrame of that group's data.
    """
    df = pd.read_excel(filepath, header=None, engine='openpyxl')
    groups = {}
    current_timestamp = None
    current_columns = None
    data_rows = []

    for _, row in df.iterrows():
        cell_val = str(row[0]) if pd.notnull(row[0]) else ""
        if cell_val.startswith("Top 30 Stocks of "):
            if current_timestamp is not None and current_columns is not None and data_rows:
                groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
            current_timestamp = cell_val.split("Top 30 Stocks of ")[1].strip()
            current_columns = None
            data_rows = []
        elif current_timestamp is not None and current_columns is None:
            current_columns = row.dropna().tolist()
        elif current_timestamp is not None and current_columns is not None:
            if not row.isnull().all():
                row_data = row.tolist()[:len(current_columns)]
                data_rows.append(row_data)
    if current_timestamp is not None and current_columns is not None and data_rows:
        groups[current_timestamp] = pd.DataFrame(data_rows, columns=current_columns)
    return groups

# Read Ranking 21 file.
ranking21_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 21.xlsx'
ranking21_groups = parse_ranking_file(ranking21_path)

# Extract summary records: one row per Timestamp per model.
summary_records = []
block_width = 7  # each ranking block is 7 columns

for ts, df_group in ranking21_groups.items():
    if df_group.shape[0] < 1:
        continue
    total_row = df_group.iloc[-1]  # last row is assumed to be the Total Return row
    ncols = df_group.shape[1]
    nblocks = ncols // block_width  # number of ranking blocks in the group
    for i in range(nblocks):
        block_start = i * block_width
        # Extract the model name from the header of the Rank column (column index block_start + 1)
        model_name = df_group.columns[block_start + 1]
        model_name = model_name.replace(" Rank", "").strip()
        # Extract the Total Return from the Return column (column index block_start + 2) using iloc.
        total_return = total_row.iloc[block_start + 2]
        summary_records.append({
            "Timestamp": ts,
            "Model": model_name,
            "Total Return": total_return
        })

# Create a summary DataFrame.
df_summary = pd.DataFrame(summary_records)
# Ensure the Timestamp column is a stripped string.
df_summary["Timestamp"] = df_summary["Timestamp"].astype(str).str.strip()
df_summary = df_summary.sort_values("Timestamp")

# Save the summary DataFrame as an Excel file "Ranking 22.xlsx".
output_excel = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 22.xlsx'
with pd.ExcelWriter(output_excel, engine='xlsxwriter') as writer:
    df_summary.to_excel(writer, index=False, sheet_name="Summary")

print(f"Ranking 22 successfully saved to {output_excel}")


Ranking 22 successfully saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 22.xlsx


In [ ]:
# Install required packages if not already installed.
!pip install openpyxl plotly

import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# Read the summary Excel file (Ranking 22).
summary_excel = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 22.xlsx'
df_summary = pd.read_excel(summary_excel)
# Ensure Timestamps are strings and stripped.
df_summary["Timestamp"] = df_summary["Timestamp"].astype(str).str.strip()

# Generate a full list of timestamps from 200601 to 202312.
full_periods = pd.period_range(start='2006-01', end='2023-12', freq='M')
full_timestamps = [p.strftime('%Y%m') for p in full_periods]

# Build traces: one trace per model.
models = df_summary["Model"].unique()
traces = []
for model in models:
    df_model = df_summary[df_summary["Model"] == model]
    # Create a mapping from Timestamp to Total Return.
    ret_dict = dict(zip(df_model["Timestamp"], df_model["Total Return"]))
    # Build y-values in order of the full_timestamps list.
    y_vals = [ret_dict.get(ts, None) for ts in full_timestamps]
    traces.append(go.Scatter(
        x=full_timestamps,
        y=y_vals,
        mode='lines+markers',
        name=model
    ))

# Create the figure.
fig = go.Figure(
    data=traces,
    layout=go.Layout(
        title="Total Return by Model Over Time (200601 to 202312)",
        xaxis=dict(title="Timestamp"),
        yaxis=dict(title="Total Return")
    )
)

# Configuration: add full-screen toggle button.
config = {
    'modeBarButtonsToAdd': ['toggleFullScreen'],
    'displayModeBar': True,
    'responsive': True
}

# Save the chart as an HTML file "Chart 3.html".
output_html = '/content/drive/My Drive/Colab Notebooks/823 Project/Chart 3.html'
pio.write_html(fig, file=output_html, auto_open=True, config=config)

print(f"Chart saved to {output_html}. Open this file in a web browser for full-screen functionality.")


Chart saved to /content/drive/My Drive/Colab Notebooks/823 Project/Chart 3.html. Open this file in a web browser for full-screen functionality.


In [ ]:
# Install required packages.
!pip install yfinance openpyxl

import yfinance as yf
import pandas as pd

# Define the benchmark tickers and friendly names.
tickers = {
    "S&P500": "^GSPC",
    "DowJones": "^DJI",
    "Nasdaq": "^IXIC",
    "SPTSX": "^GSPTSE"
}

# Download monthly data from 2006-01-01 to 2023-12-31.
data = yf.download(list(tickers.values()), start="2006-01-01", end="2023-12-31", interval="1mo")

# Use the Adjusted Close prices if available, otherwise fallback to Close.
if "Adj Close" in data.columns:
    try:
        adj_close = data["Adj Close"]
    except KeyError:
        adj_close = data.xs("Adj Close", axis=1, level=0)
elif isinstance(data.columns, pd.MultiIndex) and "Adj Close" in data.columns.get_level_values(0):
    adj_close = data.xs("Adj Close", axis=1, level=0)
else:
    if "Close" in data.columns:
        adj_close = data["Close"]
    elif isinstance(data.columns, pd.MultiIndex) and "Close" in data.columns.get_level_values(0):
        adj_close = data.xs("Close", axis=1, level=0)
    else:
        raise KeyError("Neither 'Adj Close' nor 'Close' found in downloaded data.")

# Calculate monthly returns.
monthly_return = adj_close.pct_change()

# Calculate cumulative total return for each index.
cumulative_return = (1 + monthly_return).cumprod() - 1

# Format the date index as YYYYMM.
cumulative_return.index = cumulative_return.index.strftime('%Y%m')

# Rename columns to friendly names.
mapping = {v: k for k, v in tickers.items()}
cumulative_return = cumulative_return.rename(columns=mapping)

# Reset index and rename the first column to "Timestamp".
df_temp = cumulative_return.reset_index()
df_temp = df_temp.rename(columns={df_temp.columns[0]: "Timestamp"})

# Melt the DataFrame to long format using "Timestamp" as the id variable.
df_bench = df_temp.melt(id_vars="Timestamp", var_name="Index", value_name="Total Return")

# Filter the period from 200601 to 202312.
df_bench = df_bench[(df_bench["Timestamp"] >= "200601") & (df_bench["Timestamp"] <= "202312")]
df_bench = df_bench.sort_values("Timestamp")

# Save the benchmark returns as Ranking 23.xlsx.
output_excel_bench = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 23.xlsx'
df_bench.to_excel(output_excel_bench, index=False)
print("Ranking 23 saved to", output_excel_bench)


[*********************100%***********************]  4 of 4 completed


Ranking 23 saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 23.xlsx


In [ ]:
# Install required packages.
!pip install yfinance openpyxl

import yfinance as yf
import pandas as pd

# Define the benchmark tickers and friendly names.
tickers = {
    "S&P500": "^GSPC",
    "DowJones": "^DJI",
    "Nasdaq": "^IXIC",
    "SPTSX": "^GSPTSE"
}

# Download monthly data from December 2005 to December 2023.
data = yf.download(list(tickers.values()), start="2005-12-01", end="2023-12-31", interval="1mo")

# Use the Adjusted Close prices if available, otherwise fallback to Close.
if "Adj Close" in data.columns:
    try:
        adj_close = data["Adj Close"]
    except KeyError:
        adj_close = data.xs("Adj Close", axis=1, level=0)
elif isinstance(data.columns, pd.MultiIndex) and "Adj Close" in data.columns.get_level_values(0):
    adj_close = data.xs("Adj Close", axis=1, level=0)
else:
    if "Close" in data.columns:
        adj_close = data["Close"]
    elif isinstance(data.columns, pd.MultiIndex) and "Close" in data.columns.get_level_values(0):
        adj_close = data.xs("Close", axis=1, level=0)
    else:
        raise KeyError("Neither 'Adj Close' nor 'Close' found in downloaded data.")

# Calculate monthly returns.
monthly_return = adj_close.pct_change()

# Calculate cumulative total return for each index.
cumulative_return = (1 + monthly_return).cumprod() - 1

# Format the date index as YYYYMM.
cumulative_return.index = cumulative_return.index.strftime('%Y%m')

# Rename columns to friendly names.
mapping = {v: k for k, v in tickers.items()}
cumulative_return = cumulative_return.rename(columns=mapping)

# Reset index and rename it to "Timestamp".
df_temp = cumulative_return.reset_index().rename(columns={cumulative_return.reset_index().columns[0]: "Timestamp"})

# Melt the DataFrame to long format using "Timestamp" as the id variable.
df_bench = df_temp.melt(id_vars="Timestamp", var_name="Index", value_name="Total Return")

# Filter the period from 200512 to 202312.
df_bench = df_bench[(df_bench["Timestamp"] >= "200512") & (df_bench["Timestamp"] <= "202312")]
df_bench = df_bench.sort_values("Timestamp")

# Save the benchmark returns as Ranking 23.xlsx.
output_excel_bench = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 23.xlsx'
df_bench.to_excel(output_excel_bench, index=False)
print("Ranking 23 saved to", output_excel_bench)


[*********************100%***********************]  4 of 4 completed

Ranking 23 saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 23.xlsx


In [ ]:
# Install required packages if not already installed.
!pip install openpyxl

import pandas as pd

# Read the portfolio returns file (Ranking 22).
ranking22_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 22.xlsx'
df_portfolio = pd.read_excel(ranking22_path)
df_portfolio["Timestamp"] = df_portfolio["Timestamp"].astype(str).str.strip()
# Convert the Total Return column to numeric.
df_portfolio["Total Return"] = pd.to_numeric(df_portfolio["Total Return"], errors="coerce")

# Read the benchmark returns file (Ranking 23).
ranking23_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 23.xlsx'
df_bench = pd.read_excel(ranking23_path)
df_bench["Timestamp"] = df_bench["Timestamp"].astype(str).str.strip()
df_bench["Total Return"] = pd.to_numeric(df_bench["Total Return"], errors="coerce")

# Rename the "Index" column in the benchmark file to "Model" for consistency.
df_bench = df_bench.rename(columns={"Index": "Model"})

# Combine the two DataFrames vertically.
df_combined = pd.concat([df_portfolio, df_bench], ignore_index=True)

# Sort the combined DataFrame by Timestamp.
df_combined = df_combined.sort_values("Timestamp")

# Save the combined file as Ranking 24.xlsx.
output_excel = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 24.xlsx'
df_combined.to_excel(output_excel, index=False)
print("Ranking 24 saved to", output_excel)


Ranking 24 saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 24.xlsx


In [ ]:
# Install required packages if not already installed.
!pip install openpyxl plotly

import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# ---------------------------
# Step 1. Read Ranking 24 and Prepare Data
# ---------------------------
ranking24_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 24.xlsx'
df = pd.read_excel(ranking24_path)
# Ensure that the Timestamp column is a stripped string and sort the data.
df["Timestamp"] = df["Timestamp"].astype(str).str.strip()
df = df.sort_values("Timestamp")

# ---------------------------
# Step 2. Build Full Timestamp Range and Traces for Each Model
# ---------------------------
# Create a full list of timestamps from December 2005 to December 2023.
full_periods = pd.period_range(start='2005-12', end='2023-12', freq='M')
full_timestamps = [p.strftime('%Y%m') for p in full_periods]

# Get the unique models (which now include both portfolio and benchmark returns).
models = df["Model"].unique()

traces = []
for model in models:
    df_model = df[df["Model"] == model]
    # Create a mapping from Timestamp to Total Return.
    ret_dict = dict(zip(df_model["Timestamp"], df_model["Total Return"]))
    # Build y-values for every timestamp in the full range.
    y_vals = [ret_dict.get(ts, None) for ts in full_timestamps]
    traces.append(go.Scatter(
        x=full_timestamps,
        y=y_vals,
        mode='lines+markers',
        name=model
    ))

# ---------------------------
# Step 3. Create the Chart with Full-Screen Capability
# ---------------------------
fig = go.Figure(
    data=traces,
    layout=go.Layout(
        title="Combined Portfolio & Benchmark Total Return Over Time (200512 to 202312)",
        xaxis=dict(title="Timestamp"),
        yaxis=dict(title="Total Return")
    )
)

# Configuration to add a full-screen toggle button.
config = {
    'modeBarButtonsToAdd': ['toggleFullScreen'],
    'displayModeBar': True,
    'responsive': True
}

# Save the chart as a standalone HTML file "Chart 4.html".
output_html = '/content/drive/My Drive/Colab Notebooks/823 Project/Chart 4.html'
pio.write_html(fig, file=output_html, auto_open=True, config=config)

print("Chart saved to", output_html)


Chart saved to /content/drive/My Drive/Colab Notebooks/823 Project/Chart 4.html


In [ ]:
# Install required packages.
!pip install yfinance openpyxl

import yfinance as yf
import pandas as pd

# Define the benchmark tickers and friendly names.
tickers = {
    "S&P500": "^GSPC",
    "DowJones": "^DJI",
    "Nasdaq": "^IXIC",
    "SPTSX": "^GSPTSE"
}

# Download monthly data from December 2005 to December 2023.
data = yf.download(list(tickers.values()), start="2005-12-01", end="2023-12-31", interval="1mo")

# Use Adjusted Close prices if available, otherwise fallback to Close.
if "Adj Close" in data.columns:
    try:
        adj_close = data["Adj Close"]
    except KeyError:
        adj_close = data.xs("Adj Close", axis=1, level=0)
elif isinstance(data.columns, pd.MultiIndex) and "Adj Close" in data.columns.get_level_values(0):
    adj_close = data.xs("Adj Close", axis=1, level=0)
else:
    if "Close" in data.columns:
        adj_close = data["Close"]
    elif isinstance(data.columns, pd.MultiIndex) and "Close" in data.columns.get_level_values(0):
        adj_close = data.xs("Close", axis=1, level=0)
    else:
        raise KeyError("Neither 'Adj Close' nor 'Close' found in downloaded data.")

# Calculate monthly percentage returns.
monthly_return = adj_close.pct_change()

# Format the date index as YYYYMM.
monthly_return.index = monthly_return.index.strftime('%Y%m')

# Rename columns to friendly names.
mapping = {v: k for k, v in tickers.items()}
monthly_return = monthly_return.rename(columns=mapping)

# Reset index and rename the first column to "Timestamp".
df_temp_return = monthly_return.reset_index().rename(columns={monthly_return.reset_index().columns[0]: "Timestamp"})

# Melt the monthly_return DataFrame to long format using "Timestamp" as the id variable.
df_return = df_temp_return.melt(id_vars="Timestamp", var_name="Index Name", value_name="Total Return")

# Prepare raw index readings with the same timestamp format.
df_reading = adj_close.copy()
df_reading.index = df_reading.index.strftime('%Y%m')
df_reading = df_reading.rename(columns=mapping)

# Reset index and rename the first column to "Timestamp".
df_temp_raw = df_reading.reset_index().rename(columns={df_reading.reset_index().columns[0]: "Timestamp"})

# Melt the raw index readings into long format.
df_raw = df_temp_raw.melt(id_vars="Timestamp", var_name="Index Name", value_name="Index")

# Merge the two DataFrames on Timestamp and Index Name.
df_merged = pd.merge(df_return, df_raw, on=["Timestamp", "Index Name"], how="left")

# For the final output, keep columns: Timestamp, Index Name, Total Return, and Index.
df_final = df_merged[["Timestamp", "Index Name", "Total Return", "Index"]].copy()

# Filter the period from 200512 to 202312.
df_final = df_final[(df_final["Timestamp"] >= "200512") & (df_final["Timestamp"] <= "202312")]
df_final = df_final.sort_values(["Index Name", "Timestamp"])

# Save the file as Ranking 25.xlsx.
output_excel = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 25.xlsx'
df_final.to_excel(output_excel, index=False)
print("Ranking 25 saved to", output_excel)


[*********************100%***********************]  4 of 4 completed


Ranking 25 saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 25.xlsx


In [ ]:
import pandas as pd

# ---------------------------
# Step 1. Read Ranking 22 and Ranking 25
# ---------------------------
# Path to Ranking 22 (portfolio returns: Actual & model-based returns)
ranking22_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 22.xlsx'
df22 = pd.read_excel(ranking22_path)
# Expected columns in df22: "Timestamp", "Model", "Total Return"
df22["Timestamp"] = df22["Timestamp"].astype(str).str.strip()

# Path to Ranking 25 (benchmark returns from indices)
ranking25_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 25.xlsx'
df25 = pd.read_excel(ranking25_path)
# Expected columns in df25: "Timestamp", "Index Name", "Total Return", "Index"
df25["Timestamp"] = df25["Timestamp"].astype(str).str.strip()

# ---------------------------
# Step 2. Standardize Column Names
# ---------------------------
# For Ranking 22: rename "Model" -> "Model/Index", "Total Return" -> "Total Monthly Return"
df22_std = df22[['Timestamp', 'Model', 'Total Return']].copy()
df22_std = df22_std.rename(columns={'Model': 'Model/Index', 'Total Return': 'Total Monthly Return'})

# For Ranking 25: rename "Index Name" -> "Model/Index", "Total Return" -> "Total Monthly Return"
df25_std = df25[['Timestamp', 'Index Name', 'Total Return']].copy()
df25_std = df25_std.rename(columns={'Index Name': 'Model/Index', 'Total Return': 'Total Monthly Return'})

# ---------------------------
# Step 3. Combine the Two DataFrames Vertically
# ---------------------------
df_combined = pd.concat([df22_std, df25_std], ignore_index=True)

# Ensure "Total Monthly Return" is numeric.
df_combined['Total Monthly Return'] = pd.to_numeric(df_combined['Total Monthly Return'], errors='coerce')

# ---------------------------
# Step 4. Sort the Data
# ---------------------------
# First sort by Timestamp (assumed to be in YYYYMM format)
df_combined = df_combined.sort_values(by='Timestamp')

# Define the desired model order.
desired_order = [
    "Actual", "S&P500", "DowJones", "Nasdaq", "SPTSX",
    "RollingOLS Single-scaled 1", "RollingOLS Single-scaled 2",
    "Logistic Single-scaled", "RollingOLS Double-scaled",
    "Logistic Double-scaled", "Random Forest"
]

# Convert the "Model/Index" column to a categorical type with the specified order.
df_combined['Model/Index'] = pd.Categorical(df_combined['Model/Index'], categories=desired_order, ordered=True)

# Now sort by Timestamp and then by Model/Index.
df_combined = df_combined.sort_values(by=['Timestamp', 'Model/Index'])

# ---------------------------
# Step 5. Save the Combined File as Ranking 26.xlsx
# ---------------------------
output_excel = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 26.xlsx'
df_combined.to_excel(output_excel, index=False)
print("Ranking 26 saved to", output_excel)


Ranking 26 saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 26.xlsx


In [ ]:
# Install required packages.
!pip install yfinance openpyxl

import yfinance as yf
import pandas as pd

# Define the benchmark tickers and friendly names.
tickers = {
    "S&P500": "^GSPC",
    "DowJones": "^DJI",
    "Nasdaq": "^IXIC",
    "SPTSX": "^GSPTSE"
}

# Download monthly data from December 2005 to December 2023.
data = yf.download(list(tickers.values()), start="2005-12-01", end="2023-12-31", interval="1mo")

# Use Adjusted Close prices if available, otherwise fallback to Close.
if "Adj Close" in data.columns:
    try:
        adj_close = data["Adj Close"]
    except KeyError:
        adj_close = data.xs("Adj Close", axis=1, level=0)
elif isinstance(data.columns, pd.MultiIndex) and "Adj Close" in data.columns.get_level_values(0):
    adj_close = data.xs("Adj Close", axis=1, level=0)
else:
    if "Close" in data.columns:
        adj_close = data["Close"]
    elif isinstance(data.columns, pd.MultiIndex) and "Close" in data.columns.get_level_values(0):
        adj_close = data.xs("Close", axis=1, level=0)
    else:
        raise KeyError("Neither 'Adj Close' nor 'Close' found in downloaded data.")

# Calculate monthly percentage returns and multiply by 100 for percentage.
monthly_return = adj_close.pct_change() * 100

# Format the date index as YYYYMM.
monthly_return.index = monthly_return.index.strftime('%Y%m')

# Rename columns to friendly names.
mapping = {v: k for k, v in tickers.items()}
monthly_return = monthly_return.rename(columns=mapping)

# Reset index and rename the first column to "Timestamp".
df_temp_return = monthly_return.reset_index().rename(columns={monthly_return.reset_index().columns[0]: "Timestamp"})

# Melt the monthly_return DataFrame to long format using "Timestamp" as the id variable.
df_return = df_temp_return.melt(id_vars="Timestamp", var_name="Index Name", value_name="Total Return")

# Prepare raw index readings with the same timestamp format.
df_reading = adj_close.copy()
df_reading.index = df_reading.index.strftime('%Y%m')
df_reading = df_reading.rename(columns=mapping)

# Reset index and rename the first column to "Timestamp".
df_temp_raw = df_reading.reset_index().rename(columns={df_reading.reset_index().columns[0]: "Timestamp"})

# Melt the raw index readings into long format.
df_raw = df_temp_raw.melt(id_vars="Timestamp", var_name="Index Name", value_name="Index")

# Merge the two DataFrames on Timestamp and Index Name.
df_merged = pd.merge(df_return, df_raw, on=["Timestamp", "Index Name"], how="left")

# For the final output, keep columns: Timestamp, Index Name, Total Return, and Index.
df_final = df_merged[["Timestamp", "Index Name", "Total Return", "Index"]].copy()

# Filter the period from 200512 to 202312.
df_final = df_final[(df_final["Timestamp"] >= "200512") & (df_final["Timestamp"] <= "202312")]
df_final = df_final.sort_values(["Index Name", "Timestamp"])

# Save the file as Ranking 27.xlsx.
output_excel = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 27.xlsx'
df_final.to_excel(output_excel, index=False)
print("Ranking 27 saved to", output_excel)


[*********************100%***********************]  4 of 4 completed


Ranking 27 saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 27.xlsx


In [ ]:
import pandas as pd

# ---------------------------
# Step 1. Read Ranking 22 (Portfolio Returns)
# ---------------------------
ranking22_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 22.xlsx'
df22 = pd.read_excel(ranking22_path)
# Ensure Timestamp is a string.
df22["Timestamp"] = df22["Timestamp"].astype(str).str.strip()

# Standardize column names:
# Assume Ranking 22 originally has columns: "Timestamp", "Model", "Total Return".
# We rename "Model" to "Model/Index" and "Total Return" to "Total Monthly Return".
df22_std = df22[['Timestamp', 'Model', 'Total Return']].copy()
df22_std = df22_std.rename(columns={'Model': 'Model/Index', 'Total Return': 'Total Monthly Return'})

# ---------------------------
# Step 2. Read Ranking 27 (Benchmark Returns)
# ---------------------------
ranking27_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 27.xlsx'
df27 = pd.read_excel(ranking27_path)
# Ensure Timestamp is a string.
df27["Timestamp"] = df27["Timestamp"].astype(str).str.strip()

# Ranking 27 has columns: "Timestamp", "Index Name", "Total Return", "Index"
# We rename "Index Name" to "Model/Index" and "Total Return" to "Total Monthly Return".
df27_std = df27[['Timestamp', 'Index Name', 'Total Return']].copy()
df27_std = df27_std.rename(columns={'Index Name': 'Model/Index', 'Total Return': 'Total Monthly Return'})

# ---------------------------
# Step 3. Combine the DataFrames Vertically
# ---------------------------
df_combined = pd.concat([df22_std, df27_std], ignore_index=True)

# Ensure Total Monthly Return is numeric.
df_combined['Total Monthly Return'] = pd.to_numeric(df_combined['Total Monthly Return'], errors='coerce')

# ---------------------------
# Step 4. Sort the Data
# ---------------------------
# First, sort by Timestamp (assumed to be in YYYYMM format).
df_combined = df_combined.sort_values(by='Timestamp')

# Define the desired order for the Model/Index column.
desired_order = [
    "Actual", "S&P500", "DowJones", "Nasdaq", "SPTSX",
    "RollingOLS Single-scaled 1", "RollingOLS Single-scaled 2",
    "Logistic Single-scaled", "RollingOLS Double-scaled",
    "Logistic Double-scaled", "Random Forest"
]

# Convert the Model/Index column to a categorical type with the specified order.
df_combined['Model/Index'] = pd.Categorical(df_combined['Model/Index'], categories=desired_order, ordered=True)

# Now sort by Timestamp and then by Model/Index.
df_combined = df_combined.sort_values(by=['Timestamp', 'Model/Index'])

# ---------------------------
# Step 5. Save the Combined File as Ranking 28.xlsx
# ---------------------------
output_excel = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 28.xlsx'
df_combined.to_excel(output_excel, index=False)
print("Ranking 28 saved to", output_excel)


Ranking 28 saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 28.xlsx


In [ ]:
# Install required packages if not already installed.
!pip install openpyxl plotly

import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# ---------------------------
# Step 1. Read Ranking 28 and Prepare Data
# ---------------------------
ranking28_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 28.xlsx'
df = pd.read_excel(ranking28_path)

# Ensure that the Timestamp column is a string and sorted.
df["Timestamp"] = df["Timestamp"].astype(str).str.strip()
df = df.sort_values("Timestamp")

# ---------------------------
# Step 2. Create a Full Monthly Timeline
# ---------------------------
# Here, we generate timestamps from December 2005 to December 2023.
full_periods = pd.period_range(start='2005-12', end='2023-12', freq='M')
full_timestamps = [p.strftime('%Y%m') for p in full_periods]

# ---------------------------
# Step 3. Build Traces for Each Model/Index
# ---------------------------
models = df["Model/Index"].unique()
traces = []
for model in models:
    df_model = df[df["Model/Index"] == model]
    # Create a mapping from Timestamp to Total Monthly Return.
    ret_dict = dict(zip(df_model["Timestamp"], df_model["Total Monthly Return"]))
    # Build y-values in order of the full timeline.
    y_vals = [ret_dict.get(ts, None) for ts in full_timestamps]
    traces.append(go.Scatter(
        x = full_timestamps,
        y = y_vals,
        mode = 'lines+markers',
        name = model
    ))

# ---------------------------
# Step 4. Create the Chart with Full-Screen Capability
# ---------------------------
fig = go.Figure(
    data=traces,
    layout=go.Layout(
        title="Total Monthly Return by Portfolio & Benchmark Over Time (Ranking 28)",
        xaxis=dict(title="Timestamp"),
        yaxis=dict(title="Total Monthly Return (%)")
    )
)

# Add full-screen toggle in the mode bar.
config = {
    'modeBarButtonsToAdd': ['toggleFullScreen'],
    'displayModeBar': True,
    'responsive': True
}

# ---------------------------
# Step 5. Save the Chart as "Chart 5.html"
# ---------------------------
output_html = '/content/drive/My Drive/Colab Notebooks/823 Project/Chart 5.html'
pio.write_html(fig, file=output_html, auto_open=True, config=config)
print("Chart saved to", output_html)


Chart saved to /content/drive/My Drive/Colab Notebooks/823 Project/Chart 5.html


In [ ]:
# Install required packages if not already installed.
!pip install openpyxl plotly

import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# ---------------------------
# Step 1. Read Ranking 28 and Prepare Data
# ---------------------------
ranking28_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 28.xlsx'
df = pd.read_excel(ranking28_path)
df["Timestamp"] = df["Timestamp"].astype(str).str.strip()
df = df.sort_values("Timestamp")

# ---------------------------
# Step 2. Build a Full Monthly Timeline
# ---------------------------
# Assuming the timeline covers from 200512 to 202312.
full_periods = pd.period_range(start='2005-12', end='2023-12', freq='M')
full_timestamps = [p.strftime('%Y%m') for p in full_periods]

# ---------------------------
# Step 3. Desired Order and Color Mapping
# ---------------------------
desired_order = [
    "Actual", "S&P500", "DowJones", "Nasdaq", "SPTSX",
    "RollingOLS Single-scaled 1", "RollingOLS Single-scaled 2",
    "Logistic Single-scaled", "RollingOLS Double-scaled",
    "Logistic Double-scaled", "Random Forest"
]

# Define distinctive colors for each series.
color_mapping = {
    "Actual": "#000000",                        # Black
    "S&P500": "#1f77b4",                        # Blue
    "DowJones": "#ff7f0e",                      # Orange
    "Nasdaq": "#2ca02c",                        # Green
    "SPTSX": "#d62728",                         # Red
    "RollingOLS Single-scaled 1": "#9467bd",      # Purple
    "RollingOLS Single-scaled 2": "#8c564b",      # Brown
    "Logistic Single-scaled": "#e377c2",          # Pink
    "RollingOLS Double-scaled": "#7f7f7f",        # Grey
    "Logistic Double-scaled": "#bcbd22",          # Olive/Yellow-green
    "Random Forest": "#17becf"                   # Cyan
}

# ---------------------------
# Step 4. Build Traces for Each Series in Desired Order
# ---------------------------
traces = []
for model in desired_order:
    # Filter the data for the current model.
    df_model = df[df["Model/Index"] == model]
    # Create a mapping from Timestamp to Total Monthly Return.
    ret_dict = dict(zip(df_model["Timestamp"], df_model["Total Monthly Return"]))
    # Build y-values in order of the full timeline.
    y_vals = [ret_dict.get(ts, None) for ts in full_timestamps]
    traces.append(go.Scatter(
        x = full_timestamps,
        y = y_vals,
        mode = 'lines+markers',
        name = model,
        line = dict(color = color_mapping.get(model, "#000000"))
    ))

# ---------------------------
# Step 5. Create the Chart with Full-Screen Capability
# ---------------------------
fig = go.Figure(
    data = traces,
    layout = go.Layout(
        title = "Total Monthly Portfolio Returns over Time (2006 Jan - 2023 Dec)",
        xaxis = dict(title="Timestamp"),
        yaxis = dict(title="Total Monthly Return (%)")
    )
)

# Configuration: add full-screen toggle button.
config = {
    'modeBarButtonsToAdd': ['toggleFullScreen'],
    'displayModeBar': True,
    'responsive': True
}

# Save the chart as "Chart 6.html".
output_html = '/content/drive/My Drive/Colab Notebooks/823 Project/Chart 6.html'
pio.write_html(fig, file=output_html, auto_open=True, config=config)

print("Chart saved to", output_html)


Chart saved to /content/drive/My Drive/Colab Notebooks/823 Project/Chart 6.html


In [ ]:
# Install required packages if not already installed.
!pip install openpyxl plotly

import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# ---------------------------
# Step 1. Read Ranking 28 and Prepare Data
# ---------------------------
ranking28_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 28.xlsx'
df = pd.read_excel(ranking28_path)
df["Timestamp"] = df["Timestamp"].astype(str).str.strip()
df = df.sort_values("Timestamp")

# ---------------------------
# Step 2. Build a Full Monthly Timeline
# ---------------------------
# Generate a full timeline from December 2005 to December 2023.
full_periods = pd.period_range(start='2005-12', end='2023-12', freq='M')
full_timestamps = [p.strftime('%Y%m') for p in full_periods]

# ---------------------------
# Step 3. Desired Order and Softer Color Mapping
# ---------------------------
desired_order = [
    "Actual", "S&P500", "DowJones", "Nasdaq", "SPTSX",
    "RollingOLS Single-scaled 1", "RollingOLS Single-scaled 2",
    "Logistic Single-scaled", "RollingOLS Double-scaled",
    "Logistic Double-scaled", "Random Forest"
]

# Softer pastel colors for each series:
color_mapping = {
    "Actual": "#A6A6A6",                         # Soft gray
    "S&P500": "#AEC7E8",                         # Light blue
    "DowJones": "#98DF8A",                       # Soft green
    "Nasdaq": "#FF9896",                         # Soft red/pink
    "SPTSX": "#FFBB78",                          # Soft orange
    "RollingOLS Single-scaled 1": "#C5B0D5",      # Soft purple
    "RollingOLS Single-scaled 2": "#C49C94",      # Soft brownish
    "Logistic Single-scaled": "#F7B6D2",          # Soft pink
    "RollingOLS Double-scaled": "#DBDB8D",        # Soft yellow-green
    "Logistic Double-scaled": "#9EDAE5",          # Soft teal
    "Random Forest": "#D7B5A6"                    # Soft salmon
}

# ---------------------------
# Step 4. Build Traces for Each Series in Desired Order
# ---------------------------
traces = []
for model in desired_order:
    # Filter the data for the current model.
    df_model = df[df["Model/Index"] == model]
    # Create a mapping from Timestamp to Total Monthly Return.
    ret_dict = dict(zip(df_model["Timestamp"], df_model["Total Monthly Return"]))
    # Build y-values for every timestamp in the full timeline.
    y_vals = [ret_dict.get(ts, None) for ts in full_timestamps]
    traces.append(go.Scatter(
        x = full_timestamps,
        y = y_vals,
        mode = 'lines+markers',
        name = model,
        line = dict(color = color_mapping.get(model, "#000000"))
    ))

# ---------------------------
# Step 5. Create the Chart with Full-Screen Capability and Save as "Chart 6.html"
# ---------------------------
fig = go.Figure(
    data = traces,
    layout = go.Layout(
        title = "Total Monthly Portfolio Returns over Time (2006 Jan - 2023 Dec)",
        xaxis = dict(title="Timestamp"),
        yaxis = dict(title="Total Monthly Return (%)")
    )
)

# Add full-screen toggle in the mode bar.
config = {
    'modeBarButtonsToAdd': ['toggleFullScreen'],
    'displayModeBar': True,
    'responsive': True
}

output_html = '/content/drive/My Drive/Colab Notebooks/823 Project/Chart 6.html'
pio.write_html(fig, file=output_html, auto_open=True, config=config)
print("Chart saved to", output_html)


Chart saved to /content/drive/My Drive/Colab Notebooks/823 Project/Chart 6.html


In [ ]:
# Install required packages if not already installed.
!pip install openpyxl plotly

import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# ---------------------------
# Step 1. Read Ranking 28 and Prepare Data
# ---------------------------
ranking28_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 28.xlsx'
df = pd.read_excel(ranking28_path)
df["Timestamp"] = df["Timestamp"].astype(str).str.strip()
df = df.sort_values("Timestamp")

# ---------------------------
# Step 2. Build a Full Monthly Timeline
# ---------------------------
# Generate full timeline from December 2005 to December 2023.
full_periods = pd.period_range(start='2005-12', end='2023-12', freq='M')
full_timestamps = [p.strftime('%Y%m') for p in full_periods]

# ---------------------------
# Step 3. Define Desired Order and Warm-Soft Color Mapping
# ---------------------------
desired_order = [
    "Actual", "S&P500", "DowJones", "Nasdaq", "SPTSX",
    "RollingOLS Single-scaled 1", "RollingOLS Single-scaled 2",
    "Logistic Single-scaled", "RollingOLS Double-scaled",
    "Logistic Double-scaled", "Random Forest"
]

# Warm yet soft colors:
color_mapping = {
    "Actual": "#C49E70",                   # Warm Tan
    "S&P500": "#FFA07A",                   # Light Salmon
    "DowJones": "#FF9F80",                 # Soft Warm Orange
    "Nasdaq": "#FFB6C1",                   # Light Pink
    "SPTSX": "#FFDAB9",                    # Peach Puff
    "RollingOLS Single-scaled 1": "#D8A1C4",  # Warm Lavender
    "RollingOLS Single-scaled 2": "#E6B89C",  # Soft Beige
    "Logistic Single-scaled": "#FFCC99",    # Light Orange
    "RollingOLS Double-scaled": "#CFC49E",   # Soft Olive
    "Logistic Double-scaled": "#E78A8A",     # Warm Reddish Pink
    "Random Forest": "#F08080"              # Light Coral
}

# ---------------------------
# Step 4. Build Traces for Each Series in Desired Order
# ---------------------------
traces = []
for model in desired_order:
    # Filter data for the current model.
    df_model = df[df["Model/Index"] == model]
    # Create a mapping from Timestamp to Total Monthly Return.
    ret_dict = dict(zip(df_model["Timestamp"], df_model["Total Monthly Return"]))
    # Build y-values for every timestamp in the full timeline.
    y_vals = [ret_dict.get(ts, None) for ts in full_timestamps]
    traces.append(go.Scatter(
        x = full_timestamps,
        y = y_vals,
        mode = 'lines+markers',
        name = model,
        line = dict(color = color_mapping.get(model, "#000000"))
    ))

# ---------------------------
# Step 5. Create the Chart with Full-Screen Capability and Save as "Chart 6.html"
# ---------------------------
fig = go.Figure(
    data = traces,
    layout = go.Layout(
        title = "Total Monthly Portfolio Returns over Time (2006 Jan - 2023 Dec)",
        xaxis = dict(title="Timestamp"),
        yaxis = dict(title="Total Monthly Return (%)")
    )
)

# Configuration: add full-screen toggle button.
config = {
    'modeBarButtonsToAdd': ['toggleFullScreen'],
    'displayModeBar': True,
    'responsive': True
}

output_html = '/content/drive/My Drive/Colab Notebooks/823 Project/Chart 6.html'
pio.write_html(fig, file=output_html, auto_open=True, config=config)
print("Chart saved to", output_html)


Chart saved to /content/drive/My Drive/Colab Notebooks/823 Project/Chart 6.html


In [ ]:
# Install required packages if not already installed.
!pip install openpyxl plotly

import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# ---------------------------
# Step 1. Read Ranking 28 and Prepare Data
# ---------------------------
ranking28_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 28.xlsx'
df = pd.read_excel(ranking28_path)
df["Timestamp"] = df["Timestamp"].astype(str).str.strip()
df = df.sort_values("Timestamp")

# ---------------------------
# Step 2. Build a Full Monthly Timeline (2006 Jan - 2023 Dec)
# ---------------------------
full_periods = pd.period_range(start='2006-01', end='2023-12', freq='M')
full_timestamps = [p.strftime('%Y%m') for p in full_periods]

# ---------------------------
# Step 3. Desired Order and Vibrant Color Mapping
# ---------------------------
desired_order = [
    "Actual", "S&P500", "DowJones", "Nasdaq", "SPTSX",
    "RollingOLS Single-scaled 1", "RollingOLS Single-scaled 2",
    "Logistic Single-scaled", "RollingOLS Double-scaled",
    "Logistic Double-scaled", "Random Forest"
]

# Vibrant but distinctive colors (from D3 categorical palette):
color_mapping = {
    "Actual": "#1f77b4",                        # Blue
    "S&P500": "#ff7f0e",                        # Orange
    "DowJones": "#2ca02c",                      # Green
    "Nasdaq": "#d62728",                        # Red
    "SPTSX": "#9467bd",                         # Purple
    "RollingOLS Single-scaled 1": "#8c564b",     # Brown
    "RollingOLS Single-scaled 2": "#e377c2",     # Pink
    "Logistic Single-scaled": "#7f7f7f",         # Gray
    "RollingOLS Double-scaled": "#bcbd22",       # Yellow-green
    "Logistic Double-scaled": "#17becf",         # Cyan
    "Random Forest": "#aec7e8"                   # Light Blue
}

# ---------------------------
# Step 4. Build Traces for Each Series in Desired Order
# ---------------------------
traces = []
for model in desired_order:
    df_model = df[df["Model/Index"] == model]
    ret_dict = dict(zip(df_model["Timestamp"], df_model["Total Monthly Return"]))
    y_vals = [ret_dict.get(ts, None) for ts in full_timestamps]
    traces.append(go.Scatter(
        x = full_timestamps,
        y = y_vals,
        mode = 'lines+markers',
        name = model,
        line = dict(color = color_mapping.get(model, "#000000"))
    ))

# ---------------------------
# Step 5. Create the Chart with Full-Screen Capability and Save as "Chart 7.html"
# ---------------------------
fig = go.Figure(
    data = traces,
    layout = go.Layout(
        title = "Total Monthly Portfolio Returns over Time (2006 Jan - 2023 Dec)",
        xaxis = dict(title="Timestamp"),
        yaxis = dict(title="Total Monthly Return (%)")
    )
)

config = {
    'modeBarButtonsToAdd': ['toggleFullScreen'],
    'displayModeBar': True,
    'responsive': True
}

output_html = '/content/drive/My Drive/Colab Notebooks/823 Project/Chart 7.html'
pio.write_html(fig, file=output_html, auto_open=True, config=config)
print("Chart saved to", output_html)


Chart saved to /content/drive/My Drive/Colab Notebooks/823 Project/Chart 7.html


In [ ]:
import pandas as pd

# ---------------------------
# Step 1. Read Ranking 28
# ---------------------------
ranking28_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 28.xlsx'
df28 = pd.read_excel(ranking28_path)

# Ensure the Timestamp column is a stripped string and sort by model then timestamp.
df28["Timestamp"] = df28["Timestamp"].astype(str).str.strip()
df28 = df28.sort_values(["Model/Index", "Timestamp"])

# ---------------------------
# Step 2. Create Full Timeline and Prepare Each Series
# ---------------------------
# Generate a full timeline of monthly timestamps from January 2006 to December 2023.
full_periods = pd.period_range(start='2006-01', end='2023-12', freq='M')
full_timestamps = [p.strftime('%Y%m') for p in full_periods]
timeline_df = pd.DataFrame({"Timestamp": full_timestamps})

# Prepare a list to hold cumulative return data for each series.
cumulative_list = []

# Process each series (portfolio or benchmark) separately.
for model in df28["Model/Index"].unique():
    df_model = df28[df28["Model/Index"] == model].copy()
    # Merge with full timeline so that every month is represented.
    df_model = timeline_df.merge(df_model, on="Timestamp", how="left")
    # For missing monthly returns, assume 0% (i.e. no change).
    df_model["Total Monthly Return"] = df_model["Total Monthly Return"].fillna(0)

    # Convert monthly percentage return into a growth factor:
    # Growth factor for a month = 1 + (monthly_return/100)
    # Then, the cumulative factor is the cumulative product of these monthly factors.
    df_model["Cumulative Factor"] = (1 + df_model["Total Monthly Return"] / 100).cumprod()

    # Cumulative Return (%) is then: (cumulative factor - 1) * 100.
    df_model["Cumulative Return (%)"] = (df_model["Cumulative Factor"] - 1) * 100

    # Absolute cumulative value, if $100 is invested:
    # Final value = 100 * cumulative factor.
    df_model["Cumulative Value ($)"] = 100 * df_model["Cumulative Factor"]

    # Add a column for the model name.
    df_model["Model/Index"] = model

    # Keep only the desired columns.
    cumulative_list.append(df_model[["Timestamp", "Model/Index", "Cumulative Return (%)", "Cumulative Value ($)"]])

# Combine all series.
df_cum = pd.concat(cumulative_list, ignore_index=True)
df_cum = df_cum.sort_values(by=["Timestamp", "Model/Index"])

# ---------------------------
# Step 3. Save the Output as Ranking 29.xlsx
# ---------------------------
output_excel = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 29.xlsx'
df_cum.to_excel(output_excel, index=False)
print("Ranking 29 saved to", output_excel)


Ranking 29 saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 29.xlsx


In [ ]:
# Install required packages if not already installed.
!pip install openpyxl plotly

import pandas as pd
import plotly.graph_objects as go
import plotly.io as pio

# ---------------------------
# Step 1. Read Ranking 29 and Prepare Data
# ---------------------------
ranking29_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 29.xlsx'
df29 = pd.read_excel(ranking29_path)
df29["Timestamp"] = df29["Timestamp"].astype(str).str.strip()
df29 = df29.sort_values("Timestamp")

# ---------------------------
# Step 2. Define Desired Order and Color Mapping
# ---------------------------
desired_order = [
    "Actual", "S&P500", "DowJones", "Nasdaq", "SPTSX",
    "RollingOLS Single-scaled 1", "RollingOLS Single-scaled 2",
    "Logistic Single-scaled", "RollingOLS Double-scaled",
    "Logistic Double-scaled", "Random Forest"
]

# Vibrant color mapping (from a D3 categorical palette)
color_mapping = {
    "Actual": "#1f77b4",                        # Blue
    "S&P500": "#ff7f0e",                        # Orange
    "DowJones": "#2ca02c",                      # Green
    "Nasdaq": "#d62728",                        # Red
    "SPTSX": "#9467bd",                         # Purple
    "RollingOLS Single-scaled 1": "#8c564b",     # Brown
    "RollingOLS Single-scaled 2": "#e377c2",     # Pink
    "Logistic Single-scaled": "#7f7f7f",         # Gray
    "RollingOLS Double-scaled": "#bcbd22",       # Yellow-green
    "Logistic Double-scaled": "#17becf",         # Cyan
    "Random Forest": "#aec7e8"                   # Light Blue
}

# ---------------------------
# Step 3. Build a Full Monthly Timeline
# ---------------------------
# Generate a full timeline from January 2006 to December 2023.
full_periods = pd.period_range(start='2006-01', end='2023-12', freq='M')
full_timestamps = [p.strftime('%Y%m') for p in full_periods]

# ---------------------------
# Chart 8: Build Traces for "Cumulative Return (%)"
# ---------------------------
traces_8 = []
for model in desired_order:
    df_model = df29[df29["Model/Index"] == model]
    # Create a mapping from Timestamp to Cumulative Return (%)
    ret_dict = dict(zip(df_model["Timestamp"], df_model["Cumulative Return (%)"]))
    # Build y-values for every timestamp in the full timeline.
    y_vals = [ret_dict.get(ts, None) for ts in full_timestamps]
    traces_8.append(go.Scatter(
        x = full_timestamps,
        y = y_vals,
        mode = 'lines+markers',
        name = model,
        line = dict(color = color_mapping.get(model, "#000000"))
    ))

fig8 = go.Figure(
    data = traces_8,
    layout = go.Layout(
        title = "Cumulative Return (%) over Time (2006 Jan - 2023 Dec)",
        xaxis = dict(title="Timestamp"),
        yaxis = dict(title="Cumulative Return (%)")
    )
)

# ---------------------------
# Chart 9: Build Traces for "Cumulative Value ($)"
# ---------------------------
traces_9 = []
for model in desired_order:
    df_model = df29[df29["Model/Index"] == model]
    # Create a mapping from Timestamp to Cumulative Value ($)
    val_dict = dict(zip(df_model["Timestamp"], df_model["Cumulative Value ($)"]))
    y_vals = [val_dict.get(ts, None) for ts in full_timestamps]
    traces_9.append(go.Scatter(
        x = full_timestamps,
        y = y_vals,
        mode = 'lines+markers',
        name = model,
        line = dict(color = color_mapping.get(model, "#000000"))
    ))

fig9 = go.Figure(
    data = traces_9,
    layout = go.Layout(
        title = "Cumulative Value ($) over Time (2006 Jan - 2023 Dec)",
        xaxis = dict(title="Timestamp"),
        yaxis = dict(title="Cumulative Value ($)")
    )
)

# ---------------------------
# Step 4. Save the Charts as HTML Files with Full-Screen Toggle
# ---------------------------
config = {
    'modeBarButtonsToAdd': ['toggleFullScreen'],
    'displayModeBar': True,
    'responsive': True
}

output_html8 = '/content/drive/My Drive/Colab Notebooks/823 Project/Chart 8.html'
pio.write_html(fig8, file=output_html8, auto_open=True, config=config)
print("Chart 8 saved to", output_html8)

output_html9 = '/content/drive/My Drive/Colab Notebooks/823 Project/Chart 9.html'
pio.write_html(fig9, file=output_html9, auto_open=True, config=config)
print("Chart 9 saved to", output_html9)


Chart 8 saved to /content/drive/My Drive/Colab Notebooks/823 Project/Chart 8.html
Chart 9 saved to /content/drive/My Drive/Colab Notebooks/823 Project/Chart 9.html


In [ ]:
import pandas as pd

# ---------------------------
# Step 1. Read Ranking 28 and Filter for Period 200512 to 202312
# ---------------------------
# Ranking 28 is assumed to have columns: Timestamp, Model/Index, Total Monthly Return (monthly %)
ranking28_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 28.xlsx'
df28 = pd.read_excel(ranking28_path)
df28["Timestamp"] = df28["Timestamp"].astype(str).str.strip()

# Use data from 200512 onward
df28 = df28[(df28["Timestamp"] >= "200512") & (df28["Timestamp"] <= "202312")]

# ---------------------------
# Step 2. Compute Cumulative Returns for Each Model
# ---------------------------
# For each model, calculate cumulative returns as follows:
# Growth Factor = 1 + (Monthly Return / 100)
# Cumulative Factor = cumulative product of Growth Factor
# CumReturn% = (Cumulative Factor - 1)*100
# CumReturn$ = 100 * Cumulative Factor (assuming $100 initial investment)
cumulative_list = []
for model in df28["Model/Index"].unique():
    df_model = df28[df28["Model/Index"] == model].copy()
    df_model = df_model.sort_values("Timestamp")
    # Compute the growth factor for each month
    df_model["Growth Factor"] = 1 + df_model["Total Monthly Return"] / 100
    # Compute cumulative factor (cumulative product)
    df_model["Cumulative Factor"] = df_model["Growth Factor"].cumprod()
    # Compute cumulative returns: percentage and absolute value
    df_model["CumReturn%"] = (df_model["Cumulative Factor"] - 1) * 100
    df_model["CumReturn$"] = 100 * df_model["Cumulative Factor"]
    cumulative_list.append(df_model[["Timestamp", "Model/Index", "CumReturn%", "CumReturn$"]])

df_cum = pd.concat(cumulative_list, ignore_index=True)

# ---------------------------
# Step 3. Pivot the Data into Wide Format
# ---------------------------
# We want one row per Timestamp and, for each model, two columns:
# e.g., "Actual CumReturn%" and "Actual CumReturn$"
df_pivot_pct = df_cum.pivot(index="Timestamp", columns="Model/Index", values="CumReturn%")
df_pivot_val = df_cum.pivot(index="Timestamp", columns="Model/Index", values="CumReturn$")

# Rename columns to add suffixes.
df_pivot_pct = df_pivot_pct.rename(columns=lambda x: f"{x} CumReturn%")
df_pivot_val = df_pivot_val.rename(columns=lambda x: f"{x} CumReturn$")

# Merge the two pivoted DataFrames.
df_wide = pd.merge(df_pivot_pct, df_pivot_val, left_index=True, right_index=True, how="outer").reset_index()

# ---------------------------
# Step 4. Reorder Columns According to the Desired Sequence
# ---------------------------
desired_order = [
    "Actual", "S&P500", "DowJones", "Nasdaq", "SPTSX",
    "RollingOLS Single-scaled 1", "RollingOLS Single-scaled 2",
    "Logistic Single-scaled", "RollingOLS Double-scaled",
    "Logistic Double-scaled", "Random Forest"
]

# Build the new column order: start with "Timestamp", then for each model, first the CumReturn% column then the CumReturn$ column.
ordered_columns = ["Timestamp"]
for model in desired_order:
    col_pct = f"{model} CumReturn%"
    col_val = f"{model} CumReturn$"
    if col_pct in df_wide.columns:
        ordered_columns.append(col_pct)
    if col_val in df_wide.columns:
        ordered_columns.append(col_val)

df_wide = df_wide[ordered_columns]

# Ensure all return columns are numeric.
for col in df_wide.columns:
    if col != "Timestamp":
        df_wide[col] = pd.to_numeric(df_wide[col], errors="coerce")

# ---------------------------
# Step 5. Save the New File as Ranking 30.xlsx
# ---------------------------
output_excel = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 30.xlsx'
df_wide.to_excel(output_excel, index=False)
print("Ranking 30 saved to", output_excel)


Ranking 30 saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 30.xlsx


In [ ]:
import pandas as pd

# ---------------------------
# Step 1. Read Ranking 28 and Filter for Period 200601 to 202312
# ---------------------------
# Ranking 28 is assumed to have columns: Timestamp, Model/Index, Total Monthly Return.
ranking28_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 28.xlsx'
df28 = pd.read_excel(ranking28_path)
df28["Timestamp"] = df28["Timestamp"].astype(str).str.strip()
# Filter period: 200601 to 202312.
df28 = df28[(df28["Timestamp"] >= "200601") & (df28["Timestamp"] <= "202312")]

# ---------------------------
# Step 2. Generate Full Timeline from 2006-01 to 2023-12
# ---------------------------
full_periods = pd.period_range(start='2006-01', end='2023-12', freq='M')
full_timestamps = [p.strftime('%Y%m') for p in full_periods]
timeline_df = pd.DataFrame({"Timestamp": full_timestamps})

# ---------------------------
# Step 3. For Each Model, Compute Cumulative Returns
# ---------------------------
cumulative_list = []
for model in df28["Model/Index"].unique():
    df_model = df28[df28["Model/Index"] == model].copy()
    df_model = df_model.sort_values("Timestamp")
    # Merge with the full timeline so every month is represented.
    df_model = timeline_df.merge(df_model, on="Timestamp", how="left")
    # For missing monthly returns, assume 0% change.
    df_model["Total Monthly Return"] = df_model["Total Monthly Return"].fillna(0)

    # Calculate growth factor for each month.
    # Growth Factor = 1 + (monthly_return/100)
    df_model["Growth Factor"] = 1 + df_model["Total Monthly Return"] / 100
    # Cumulative Factor = cumulative product of growth factors.
    df_model["Cumulative Factor"] = df_model["Growth Factor"].cumprod()
    # Cumulative Return (%) = (Cumulative Factor - 1) * 100.
    df_model["CumReturn%"] = (df_model["Cumulative Factor"] - 1) * 100
    # Cumulative Value ($) = 100 * Cumulative Factor.
    df_model["CumReturn$"] = 100 * df_model["Cumulative Factor"]

    # Rename the monthly return column to "Model Portfolio Monthly Return".
    df_model = df_model.rename(columns={"Total Monthly Return": "Model Portfolio Monthly Return"})
    # Add a column for the model name.
    df_model["Model/Index"] = model
    # Keep only needed columns.
    cumulative_list.append(df_model[["Timestamp", "Model/Index", "Model Portfolio Monthly Return", "CumReturn%", "CumReturn$"]])

df_cum = pd.concat(cumulative_list, ignore_index=True)

# ---------------------------
# Step 4. Pivot the Data into Wide Format
# ---------------------------
# We want one row per Timestamp, and for each model, three columns:
#   e.g. "Actual Monthly Return", "Actual CumReturn%", "Actual CumReturn$"
df_pivot_monthly = df_cum.pivot(index="Timestamp", columns="Model/Index", values="Model Portfolio Monthly Return")
df_pivot_pct = df_cum.pivot(index="Timestamp", columns="Model/Index", values="CumReturn%")
df_pivot_val = df_cum.pivot(index="Timestamp", columns="Model/Index", values="CumReturn$")

# Rename columns with appropriate suffixes.
df_pivot_monthly = df_pivot_monthly.rename(columns=lambda x: f"{x} Monthly Return")
df_pivot_pct = df_pivot_pct.rename(columns=lambda x: f"{x} CumReturn%")
df_pivot_val = df_pivot_val.rename(columns=lambda x: f"{x} CumReturn$")

# Merge the three pivot tables.
df_wide = pd.concat([df_pivot_monthly, df_pivot_pct, df_pivot_val], axis=1).reset_index()

# ---------------------------
# Step 5. Reorder Columns According to the Desired Sequence
# ---------------------------
desired_order = [
    "Actual", "S&P500", "DowJones", "Nasdaq", "SPTSX",
    "RollingOLS Single-scaled 1", "RollingOLS Single-scaled 2",
    "Logistic Single-scaled", "RollingOLS Double-scaled",
    "Logistic Double-scaled", "Random Forest"
]

ordered_columns = ["Timestamp"]
for model in desired_order:
    col_month = f"{model} Monthly Return"
    col_pct   = f"{model} CumReturn%"
    col_val   = f"{model} CumReturn$"
    if col_month in df_wide.columns:
        ordered_columns.append(col_month)
    if col_pct in df_wide.columns:
        ordered_columns.append(col_pct)
    if col_val in df_wide.columns:
        ordered_columns.append(col_val)

df_wide = df_wide[ordered_columns]

# Ensure all return columns are numeric.
for col in df_wide.columns:
    if col != "Timestamp":
        df_wide[col] = pd.to_numeric(df_wide[col], errors="coerce")

# ---------------------------
# Step 6. Save the Combined File as Ranking 31.xlsx
# ---------------------------
output_excel = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 31.xlsx'
df_wide.to_excel(output_excel, index=False)
print("Ranking 31 saved to", output_excel)


Ranking 31 saved to /content/drive/My Drive/Colab Notebooks/823 Project/Ranking 31.xlsx


In [50]:
!pip install python-pptx  # Ensure python-pptx is installed

from pptx import Presentation
from google.colab import drive
import shutil

# ✅ 1. Create the PowerPoint File
prs = Presentation()

# Slide 1: Journey with the Data
slide_layout = prs.slide_layouts[0]  # Title Slide
slide = prs.slides.add_slide(slide_layout)
slide.shapes.title.text = "Journey with the Data"
slide.placeholders[1].text = "Exploring Data Preparation & Transformation for Modeling"

# Slide 2: X & Y Variables
slide_layout = prs.slide_layouts[1]  # Title and Content
slide = prs.slides.add_slide(slide_layout)
slide.shapes.title.text = "X & Y Variables"

content = slide.shapes.placeholders[1].text_frame
content.text = "X Variables (Independent Features):\n" # add newline here
content.text += "- Financial Ratios (Profitability, Valuation, Leverage, Liquidity, Efficiency, Capital Allocation)\n" # append text with newline
content.text += "- Derived Metrics after PCA transformation\n" # append text with newline
content.text += "\nY Variable (Dependent Target Variable):\n" # add newline here
content.text += "- Return over the Next 3 Months" # append text with newline

# Slide 3: Data Preparation Steps
slide_layout = prs.slide_layouts[1]
slide = prs.slides.add_slide(slide_layout)
slide.shapes.title.text = "Data Preparation Overview"

content = slide.shapes.placeholders[1].text_frame
content.text = "Datasets Used & Merging:\n" # add newline here
content.text += "- Combined multiple datasets with financial ratios.\n" # append text with newline
content.text += "- Ensured consistency across financial metrics.\n" # append text with newline

content.text += "\nCleaning & EDA:\n" # add newline here
content.text += "- Identified missing values & outliers.\n" # append text with newline
content.text += "- Applied winsorization to reduce extreme outliers.\n" # append text with newline

content.text += "\nMissing Value Imputation (3 Stages):\n" # add newline here
content.text += "- Stage 1: Drop rows with missing Ticker values.\n" # append text with newline
content.text += "- Stage 2: Fill missing values with mean/median.\n" # append text with newline
content.text += "- Stage 3: Robust imputation for extreme cases.\n" # append text with newline


# Slide 4: Scaling, Multicollinearity, and PCA
slide_layout = prs.slide_layouts[1]
slide = prs.slides.add_slide(slide_layout)
slide.shapes.title.text = "Scaling, Multicollinearity & PCA"

content = slide.shapes.placeholders[1].text_frame
content.text = "Scaling (1st Stage):\n" # add newline here
content.text += "- Standardized data to zero mean and unit variance.\n" # append text with newline
content.text += "- Ensured comparability across financial ratios.\n" # append text with newline

content.text += "\nMulticollinearity Check:\n" # add newline here
content.text += "- Used Variance Inflation Factor (VIF) to detect correlated features.\n" # append text with newline
content.text += "- High-correlation features were removed or merged.\n" # append text with newline

content.text += "\nPCA Transformation:\n" # add newline here
content.text += "- Grouped 31 financial ratios into 6 categories.\n" # append text with newline
content.text += "- Applied PCA to reduce dimensionality to 10 components while preserving 95% variance.\n" # append text with newline

content.text += "\nScaling (2nd Stage):\n" # add newline here
content.text += "- Applied RobustScaler after PCA to stabilize new components.\n" # append text with newline

content.text += "\nFinal Data Readiness Check:\n" # add newline here
content.text += "- Ensured cleaned, normalized, and transformed dataset was ready for modeling.\n" # append text with newline

# ✅ Save the PowerPoint file locally in Google Colab
pptx_local_path = "/content/PPT 1.pptx"
prs.save(pptx_local_path)
print(f"✅ PowerPoint file created at: {pptx_local_path}")

# ✅ 2. Copy the File to Google Drive
drive.mount('/content/drive', force_remount=True)

# Define destination path in Google Drive
drive_path = "/content/drive/My Drive/Colab Notebooks/823 Project/PPT 1.pptx"

# Copy file to Google Drive
shutil.copy(pptx_local_path, drive_path)

print(f"✅ File successfully copied to Google Drive: {drive_path}")
print("📌 You can now find the file in your Google Drive under 'Colab Notebooks/823 Project/' and download it.")


✅ PowerPoint file created at: /content/PPT 1.pptx
Mounted at /content/drive
✅ File successfully copied to Google Drive: /content/drive/My Drive/Colab Notebooks/823 Project/PPT 1.pptx
📌 You can now find the file in your Google Drive under 'Colab Notebooks/823 Project/' and download it.


In [5]:
import pandas as pd

# Define the desired order (this should match the column naming in Ranking 31).
desired_order = [
    "Actual", "S&P500", "DowJones", "Nasdaq", "SPTSX",
    "RollingOLS Single-scaled 1", "RollingOLS Single-scaled 2",
    "Logistic Single-scaled", "RollingOLS Double-scaled",
    "Logistic Double-scaled", "Random Forest"
]

# Path to Ranking 31 file.
ranking31_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 31.xlsx'
df = pd.read_excel(ranking31_path)

# Ensure Timestamp is a string and sort the data.
df["Timestamp"] = df["Timestamp"].astype(str).str.strip()
df = df.sort_values("Timestamp")

# Initialize list to collect summary metrics for each model.
summary_list = []

for model in desired_order:
    monthly_col = f"{model} Monthly Return"
    cum_pct_col = f"{model} CumReturn%"
    cum_val_col = f"{model} CumReturn$"

    # Check if all required columns exist for the model.
    if monthly_col in df.columns and cum_pct_col in df.columns and cum_val_col in df.columns:
        # Convert columns to numeric.
        df[monthly_col] = pd.to_numeric(df[monthly_col], errors="coerce")
        df[cum_pct_col] = pd.to_numeric(df[cum_pct_col], errors="coerce")
        df[cum_val_col] = pd.to_numeric(df[cum_val_col], errors="coerce")

        # Final cumulative return: take the last value (assuming data is sorted by Timestamp).
        final_cum_pct = df.iloc[-1][cum_pct_col]
        final_cum_val = df.iloc[-1][cum_val_col]

        # Calculate average monthly return and standard deviation.
        avg_monthly = df[monthly_col].mean()
        std_monthly = df[monthly_col].std()

        summary_list.append({
            "Model": model,
            "Final CumReturn (%)": final_cum_pct,
            "Final CumReturn ($)": final_cum_val,
            "Average Monthly Return (%)": avg_monthly,
            "Monthly Return StdDev (%)": std_monthly
        })
    else:
        print(f"Columns for model '{model}' not found in Ranking 31.")

# Create a summary DataFrame.
summary_df = pd.DataFrame(summary_list)

# Sort summary_df by the desired order (based on the order in our list).
summary_df['Model'] = pd.Categorical(summary_df['Model'], categories=desired_order, ordered=True)
summary_df = summary_df.sort_values('Model')

print(summary_df)


                         Model  Final CumReturn (%)  Final CumReturn ($)  \
0                       Actual         4.960902e+10         4.960902e+10   
1                       S&P500         2.821091e+02         3.821091e+02   
2                     DowJones         2.516635e+02         3.516635e+02   
3                       Nasdaq         5.806880e+02         6.806880e+02   
4                        SPTSX         8.592834e+01         1.859283e+02   
5   RollingOLS Single-scaled 1         1.587151e+03         1.687151e+03   
6   RollingOLS Single-scaled 2         2.095939e+03         2.195939e+03   
7       Logistic Single-scaled         3.662741e+02         4.662741e+02   
8     RollingOLS Double-scaled         1.425417e+03         1.525417e+03   
9       Logistic Double-scaled         9.389594e+02         1.038959e+03   
10               Random Forest         3.087356e+04         3.097356e+04   

    Average Monthly Return (%)  Monthly Return StdDev (%)  
0                     9.880

In [6]:
import pandas as pd

# Define the desired order (this should match the ordering in Ranking 31)
desired_order = [
    "Actual", "S&P500", "DowJones", "Nasdaq", "SPTSX",
    "RollingOLS Single-scaled 1", "RollingOLS Single-scaled 2",
    "Logistic Single-scaled", "RollingOLS Double-scaled",
    "Logistic Double-scaled", "Random Forest"
]

# Path to Ranking 31 file.
ranking31_path = '/content/drive/My Drive/Colab Notebooks/823 Project/Ranking 31.xlsx'
df = pd.read_excel(ranking31_path)

# Ensure Timestamp is a string and sort the data.
df["Timestamp"] = df["Timestamp"].astype(str).str.strip()
df = df.sort_values("Timestamp")

# Create a list to hold summary metrics for each series.
summary_list = []

# For each model (or index) in our desired order, compute summary metrics.
for model in desired_order:
    # The following columns are expected in Ranking 31:
    # e.g., for "Actual", the columns should be "Actual Monthly Return", "Actual CumReturn%", "Actual CumReturn$"
    monthly_col = f"{model} Monthly Return"
    cum_pct_col = f"{model} CumReturn%"
    cum_val_col = f"{model} CumReturn$"

    # Check if these columns exist.
    if monthly_col in df.columns and cum_pct_col in df.columns and cum_val_col in df.columns:
        # Convert columns to numeric values.
        df[monthly_col] = pd.to_numeric(df[monthly_col], errors="coerce")
        df[cum_pct_col] = pd.to_numeric(df[cum_pct_col], errors="coerce")
        df[cum_val_col] = pd.to_numeric(df[cum_val_col], errors="coerce")

        # Assuming the data is sorted by Timestamp, the final cumulative values are in the last row.
        final_cum_pct = df.iloc[-1][cum_pct_col]
        final_cum_val = df.iloc[-1][cum_val_col]

        # Compute average monthly return and standard deviation (using all available months).
        avg_monthly = df[monthly_col].mean()
        std_monthly = df[monthly_col].std()

        summary_list.append({
            "Model": model,
            "Final CumReturn (%)": final_cum_pct,
            "Final CumReturn ($)": final_cum_val,
            "Average Monthly Return (%)": avg_monthly,
            "Monthly Return StdDev (%)": std_monthly
        })
    else:
        print(f"One or more columns for model '{model}' not found in Ranking 31.")

# Create a summary DataFrame.
summary_df = pd.DataFrame(summary_list)

# Convert the Model column to a categorical type for sorting in the desired order.
summary_df["Model"] = pd.Categorical(summary_df["Model"], categories=desired_order, ordered=True)
summary_df = summary_df.sort_values("Model")

# Print the summary table.
print(summary_df)


                         Model  Final CumReturn (%)  Final CumReturn ($)  \
0                       Actual         4.960902e+10         4.960902e+10   
1                       S&P500         2.821091e+02         3.821091e+02   
2                     DowJones         2.516635e+02         3.516635e+02   
3                       Nasdaq         5.806880e+02         6.806880e+02   
4                        SPTSX         8.592834e+01         1.859283e+02   
5   RollingOLS Single-scaled 1         1.587151e+03         1.687151e+03   
6   RollingOLS Single-scaled 2         2.095939e+03         2.195939e+03   
7       Logistic Single-scaled         3.662741e+02         4.662741e+02   
8     RollingOLS Double-scaled         1.425417e+03         1.525417e+03   
9       Logistic Double-scaled         9.389594e+02         1.038959e+03   
10               Random Forest         3.087356e+04         3.097356e+04   

    Average Monthly Return (%)  Monthly Return StdDev (%)  
0                     9.880